# 1. Set up iniziali

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
#1. Caricamento dati
delivery_data = pd.read_csv('delivery_history.csv').copy()

In [ ]:
#2. conversione variabile delivery_date in datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')
# Crea una seconda colonna solo con la data, tipo object (utile solo per visualizzazioni)
delivery_data['delivery_day'] = delivery_data['delivery_date'].dt.date

#3. Creazione di colonne derivate dalla data
#Giorno della settimana: 0=Monday, 6=Sunday
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek
#Escludi sabati e domeniche
# weekday_num: 0=Lun, 1=Mar, ..., 5=Sab, 6=Dom
delivery_data = delivery_data[delivery_data['weekday_num'] < 5]
delivery_data = delivery_data.dropna(subset=['delivery_day'])

#4. Mappa i numeri ai nomi (utile per grafici)
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven'}
delivery_data['weekday_name'] = delivery_data['weekday_num'].map(weekday_map)

#5. Divisione delle singole variabili temporali
# Mese (1-12)
delivery_data['month'] = delivery_data['delivery_date'].dt.month
# Anno
delivery_data['year'] = delivery_data['delivery_date'].dt.year
# Periodo (Anno-Mese) utile per analisi mensili
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

#6. Conversione is_event in booleano
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
# Applica conversione solo ai validi (opzionale: puoi forzare tutto a bool, ma questo è più sicuro)
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

# --- Check finale ---
print(delivery_data[['delivery_date', 'weekday_name', 'month', 'year', 'year_month', 'is_event']].head())
print(delivery_data.dtypes)
print(delivery_data.columns)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# 1. Plot punti lat/lon da delivery_data
plt.figure(figsize=(12, 10))
plt.scatter(delivery_data['lon'], delivery_data['lat'], s=10, alpha=0.6, label='Punti consegna')

# 2. Carica shapefile confini Lombardia (se scaricato dal geoportale Regione Lombardia)
# Ad esempio: 'Lombardia_comuni.shp' (path al file shapefile)

# 3. Plot confini Lombardia

plt.title("Punti consegna con confini regione Lombardia")
plt.xlabel("Longitudine")
plt.ylabel("Latitudine")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Conta i giorni distinti presenti per ogni mese
days_per_month = delivery_data.groupby('year_month')['delivery_day'].nunique()
# Stampa il risultato
print("Numero di giorni con dati per ogni mese:")
print(days_per_month)


## 1.1 analisi del numero di ordini mensili


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Definisci fasce precise per ordini mensili
fasce_dettagliate = {
    '1 ordine/mese': lambda x: x == 1,
    '2 ordini/mese': lambda x: x == 2,
    '3 ordini/mese': lambda x: x == 3,
    '4 ordini/mese': lambda x: x == 4,
    '5 ordini/mese': lambda x: x == 5,
    '6 ordini/mese': lambda x: x == 6,
    '7 ordini/mese': lambda x: x == 7,
    '8 ordini/mese': lambda x: x == 8,
    '9 ordini/mese': lambda x: x == 9,
    '10 ordini/mese': lambda x: x == 10,
    '11 ordini/mese': lambda x: x == 11,
    '12 o più ordini/mese': lambda x: x >=12,
}

fascia_labels_dettagliate = list(fasce_dettagliate.keys())

# Calcola i valori per ogni fascia e ogni mese (riutilizzando la variabile monthly_counts)
fasce_per_mese_dett = []
for mese in mesi:
    counts_this_month = monthly_counts[monthly_counts['year_month'] == mese]
    fasce_vals = [counts_this_month['n_orders'].apply(cond).sum() for cond in fasce_dettagliate.values()]
    fasce_per_mese_dett.append(fasce_vals)

fasce_matrix_dett = np.array(fasce_per_mese_dett)

# Converti mesi in stringhe se necessario
mesi_str = [str(m) for m in mesi]

plt.figure(figsize=(15, 8))
bottom = np.zeros(len(mesi_str))
for i, fascia in enumerate(fascia_labels_dettagliate):
    plt.bar(mesi_str, fasce_matrix_dett[:, i], label=fascia, bottom=bottom)
    for j, valore in enumerate(fasce_matrix_dett[:, i]):
        y_pos = bottom[j] + valore / 2 if valore > 0 else bottom[j]
        if valore > 0:
            plt.text(mesi_str[j], y_pos, int(valore), ha='center', va='center', fontsize=8)
    bottom += fasce_matrix_dett[:, i]

plt.xlabel('Mese')
plt.ylabel('Numero clienti (per fascia ordini)')
plt.title('Distribuzione dettagliata fasce frequenza ordini per mese')
plt.xticks(rotation=45)
plt.legend(title='Fasce ordini mensili', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 1. Filtra i mesi da luglio (7) a dicembre (12)
data_filtered = delivery_data[delivery_data['delivery_date'].dt.month.isin(range(7,13))].copy()

# Estrai anno-mese come periodo
data_filtered['year_month'] = data_filtered['delivery_date'].dt.to_period('M')

# Filtra solo ordini validi
valid_orders = data_filtered[data_filtered['is_event'] == 1]

# 2. Conta ordini mensili per cliente e mese
monthly_orders = valid_orders.groupby(['location_id', 'year_month']).size().reset_index(name='num_orders_per_month')

# Trova tutti i mesi distinti nell’intervallo (luglio-dicembre)
all_months = set(monthly_orders['year_month'].unique())
num_months = len(all_months)

# 3. Trova clienti che ordinano più di 11 volte in ogni mese (luglio-dicembre)
def find_clients_min_orders(target):
    filtered = monthly_orders[monthly_orders['num_orders_per_month'] > target]
    counts = filtered.groupby('location_id')['year_month'].nunique()
    clients = counts[counts == num_months].index.tolist()
    return clients

clients_11plus = find_clients_min_orders(10)

print(f"Clienti con più di 11 ordini in ogni mese da luglio a dicembre: {len(clients_11plus)}")

# 4. Estrai dati lat/lon di questi clienti
clients_data = delivery_data[delivery_data['location_id'].isin(clients_11plus)][['location_id', 'lat', 'lon']].drop_duplicates()
import matplotlib.pyplot as plt


plt.figure(figsize=(12, 9))
plt.scatter(clients_data['lon'], clients_data['lat'], color='red', s=50, edgecolor='black', alpha=0.7)

# Aggiungi etichette con location_id senza .0
for _, row in clients_data.iterrows():
    loc_id = str(int(row['location_id'])) if pd.notnull(row['location_id']) else ''
    plt.text(row['lon'], row['lat'], loc_id, fontsize=8, ha='right', va='bottom')

plt.title('Clienti con più di 11 ordini per mese da luglio a dicembre')
plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.grid(True)
plt.show()


In [ ]:
# Assicurati di usare il dataset con ordini validi e mesi da luglio a dicembre
# Filtra i dati per i clienti selezionati
clients_orders = valid_orders[valid_orders['location_id'].isin(clients_11plus)]

# Calcola la quantità totale ordinata da ciascun cliente
quantity_per_client = clients_orders.groupby('location_id')['quantity'].sum().reset_index(name='total_quantity')

# Se vuoi mostrare il totale complessivo (tutti clienti insieme)
total_quantity_all_clients = quantity_per_client['total_quantity'].sum()

print("Quantità totale ordinata per ciascun cliente con più di 11 ordini per mese da luglio a dicembre:")
print(quantity_per_client)

print(f"\nQuantità totale ordinata da tutti questi clienti insieme: {total_quantity_all_clients}")


In [ ]:
import matplotlib.pyplot as plt

# Filtra i mesi da luglio a dicembre
data_filtered = delivery_data[delivery_data['delivery_date'].dt.month.isin(range(7, 13))].copy()

# Aggiungi colonna anno-mese period
data_filtered['year_month'] = data_filtered['delivery_date'].dt.to_period('M')

# Filtra solo ordini validi
valid_orders = data_filtered[data_filtered['is_event'] == 1]

# Conta ordini mensili per cliente
monthly_orders = valid_orders.groupby(['location_id', 'year_month']) \
    .size().reset_index(name='num_orders_per_month')

# Prendi i mesi da luglio a dicembre nel formato year_month
mesi_lista = sorted(monthly_orders['year_month'].unique())

# Colormap (tab10 per 10 colori max, qui ne bastano 6)
cmap = plt.get_cmap('tab10', len(mesi_lista))
color_map = {mese: cmap(i) for i, mese in enumerate(mesi_lista)}

plt.figure(figsize=(12, 9))

# Per ogni mese filtra clienti che ordinano >= 11 volte e plottali in colore diverso
for mese in mesi_lista:
    clienti_mese = monthly_orders[
        (monthly_orders['year_month'] == mese) & 
        (monthly_orders['num_orders_per_month'] >= 11)
    ]['location_id'].unique()
    dati_cliente = delivery_data[
        delivery_data['location_id'].isin(clienti_mese)
    ][['location_id', 'lat', 'lon']].drop_duplicates()
    
    plt.scatter(dati_cliente['lon'], dati_cliente['lat'], 
                label=str(mese), color=color_map[mese], alpha=0.7, edgecolor='black', s=50)

plt.title('Clienti con almeno 11 ordini per mese (luglio-dicembre)')
plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.legend(title='Mese', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Usa tutto il dataset o filtra se serve
data_filtered = delivery_data.copy()

# Aggiungi colonna anno-settimana
data_filtered['year_week'] = data_filtered['delivery_date'].dt.to_period('W')

# Filtra solo ordini validi
valid_orders = data_filtered[data_filtered['is_event'] == 1]

# Conta ordini per cliente e settimana
weekly_orders = valid_orders.groupby(['location_id', 'year_week']).size().reset_index(name='num_orders_per_week')

# Calcola per ciascun cliente
client_stats = weekly_orders.groupby('location_id').apply(
    lambda df: (df['num_orders_per_week'] >= 3).mean()
).reset_index(name='freq_3plus_orders')

# Seleziona clienti con frequenza almeno 85%
clients_3plus_85 = client_stats[client_stats['freq_3plus_orders'] >= 0.85]['location_id'].unique()

# Estrai lat/lon per questi clienti
clients_data = delivery_data[delivery_data['location_id'].isin(clients_3plus_85)][['location_id', 'lat', 'lon']].drop_duplicates()

plt.figure(figsize=(12, 9))
plt.scatter(clients_data['lon'], clients_data['lat'], color='green', s=50, edgecolor='black', alpha=0.7)
for _, row in clients_data.iterrows():
    loc_id = str(int(row['location_id'])) if pd.notnull(row['location_id']) else ''
    plt.text(row['lon'], row['lat'], loc_id, fontsize=8, ha='right', va='bottom')

plt.title('Clienti che ordinano ≥3 volte a settimana per almeno l\'85% delle settimane')
plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.grid(True)
plt.show()

In [ ]:
# Filtra gli ordini validi dei clienti selezionati
clients_orders = valid_orders[valid_orders['location_id'].isin(clients_3plus_85)]

# Calcola la quantità totale ordinata per ciascun cliente
quantity_per_client = clients_orders.groupby('location_id')['quantity'].sum().reset_index(name='total_quantity')

# Stampa la tabella con location_id e quantità totale
print(quantity_per_client.to_string(index=False))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# 1. Aggiungi colonne anno-mese e anno-settimana
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')
delivery_data['year_week'] = delivery_data['delivery_date'].dt.to_period('W')

# Escludi giugno (6) e gennaio (1)
delivery_data_filtered = delivery_data[~delivery_data['delivery_date'].dt.month.isin([1,6])].copy()

# 2. Filtra solo ordini validi
orders = delivery_data_filtered[delivery_data_filtered['is_event'] == 1].copy()

# 3. Conta ordini per cliente, settimana
weekly_counts = orders.groupby(['location_id', 'year_week']).size().reset_index(name='orders_per_week')

# 4. Associa ogni settimana al mese corrispondente
week_to_month = orders.groupby('year_week')['year_month'].agg(lambda x: x.mode()[0]).to_dict()
weekly_counts['year_month'] = weekly_counts['year_week'].map(week_to_month)

# 5. Calcola per cliente e mese la media ordini a settimana
avg_weekly_orders = weekly_counts.groupby(['location_id', 'year_month'])['orders_per_week'].mean().reset_index()

# 6. Crea una colonna fascia ordini a settimana
def assign_fascia(x):
    if x < 1.5:
        return '1 ordine/settimana'
    elif x < 2.5:
        return '2 ordini/settimana'
    elif x < 3.5:
        return '3 ordini/settimana'
    else:
        return '4 o più ordini/settimana'

avg_weekly_orders['order_frequency'] = avg_weekly_orders['orders_per_week'].apply(assign_fascia)

# 7. Conta clienti per fascia e mese (clienti distinti per location_id)
clients_per_fascia = avg_weekly_orders.groupby(['year_month', 'order_frequency'])['location_id'].nunique().reset_index()

# 8. Riorganizza dati per grafico a barre impilate
pivot = clients_per_fascia.pivot(index='year_month', columns='order_frequency', values='location_id').fillna(0)

# Ordine fasce per legenda
fasce_order = ['1 ordine/settimana', '2 ordini/settimana', '3 ordini/settimana', '4 o più ordini/settimana']
pivot = pivot[fasce_order]

# 9. Grafico a barre impilate con etichette quantità
pivot.index = pivot.index.astype(str)  # per etichette x leggibili
ax = pivot.plot(kind='bar', stacked=True, figsize=(14,7), colormap='tab20')

plt.xlabel('Mese')
plt.ylabel('Numero clienti')
plt.title('Numero clienti per fascia di ordini settimanali per mese (esclusi gennaio e giugno)')
plt.xticks(rotation=45)
plt.legend(title='Frequenza ordini settimanali')
# Inserisci etichette numeriche per ogni segmento barra con posizionamento personalizzato
for i, container in enumerate(ax.containers):
    if pivot.columns[i] == '4 o più ordini/settimana':
        # Posizione etichetta un po’ più in alto per maggiore leggibilità
        ax.bar_label(container, label_type='center', padding=8, fontsize=9)
    else:
        ax.bar_label(container, label_type='center', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Filtra data senza sabato e senza location_id 14930, 15133
filtered = delivery_data[
    (delivery_data['delivery_date'].dt.weekday != 5) &  # Sabato=5
    (~delivery_data['location_id'].isin([14930, 15133]))
].copy()

# Conta ordini totali per cliente (location_id)
orders_count = filtered.groupby('location_id').size().reset_index(name='total_orders')

# Funzione per assegnare fascia in base al numero ordini (29 settimane considerate)
def assign_order_frequency(total_orders):
    if total_orders < 29:
        return 'Meno di 1 ordine/settimana'
    elif total_orders < 58:
        return '1-2 ordini/settimana'
    elif total_orders < 87:
        return '2-3 ordini/settimana'
    elif total_orders < 116:
        return '3-4 ordini/settimana'
    else:
        return 'Più di 4 ordini/settimana'

orders_count['order_frequency'] = orders_count['total_orders'].apply(assign_order_frequency)

# Conta numero clienti per fascia
clients_per_fascia = orders_count.groupby('order_frequency').size().reset_index(name='num_clients')

# Ordina le fasce in ordine logico
fasce_order = ['Meno di 1 ordine/settimana', '1-2 ordini/settimana',
               '2-3 ordini/settimana', '3-4 ordini/settimana', 'Più di 4 ordini/settimana']
clients_per_fascia['order_frequency'] = pd.Categorical(clients_per_fascia['order_frequency'], categories=fasce_order, ordered=True)
clients_per_fascia = clients_per_fascia.sort_values('order_frequency')

# Plot grafico a barre
plt.figure(figsize=(10, 6))
bars = plt.bar(clients_per_fascia['order_frequency'], clients_per_fascia['num_clients'], color=plt.cm.tab10.colors)
plt.xlabel('Fascia ordine settimanale')
plt.ylabel('Numero clienti')
plt.title('Numero clienti per frequenza ordine settimanale')

# Etichette valori sopra barre
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 1, str(height), ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Escludi gennaio (1) e giugno (6)
clients_filtered = clients_per_fascia_month[
    (~clients_per_fascia_month['delivery_date'].dt.month.isin([1, 6]))
]

# Ottieni mesi e fasce ordinate
months = sorted(clients_filtered['delivery_date'].unique())
categories = fasce_order

# Costruisci matrice dati
matrix = np.zeros((len(months), len(categories)), dtype=int)
for i, month in enumerate(months):
    for j, cat in enumerate(categories):
        cell = clients_filtered[(clients_filtered['delivery_date'] == month) & (clients_filtered['order_frequency'] == cat)]
        matrix[i, j] = cell['num_clients'].values[0] if not cell.empty else 0

# Plot barre impilate
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(months))
bottom = np.zeros(len(months))

colors = plt.cm.tab10.colors

special_labels = ['3-4 ordini/settimana', 'Più di 4 ordini/settimana']
highlight_color_map = {cat: colors[i] for i, cat in enumerate(categories) if cat in special_labels}

# Offset personalizzati
highlight_offset_map = {
    '3-4 ordini/settimana': 0,   # etichetta quasi alla base della barra
    'Più di 4 ordini/settimana': 50  # etichetta più alta per evitare sovrapposizioni
}

for i, category in enumerate(categories):
    bars = ax.bar(x, matrix[:, i], bottom=bottom, color=colors[i], label=category)
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            if category in special_labels:
                offset = highlight_offset_map.get(category, 0.5)
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + height + offset,
                        str(height), ha='center', va='bottom', fontsize=8, color=highlight_color_map[category])
            else:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + height/2,
                        str(height), ha='center', va='center', fontsize=8, color='white')
    bottom += matrix[:, i]

ax.set_xticks(x)
ax.set_xticklabels([str(m) for m in months], rotation=45)

ax.set_xlabel('Mese')
ax.set_ylabel('Numero clienti')
ax.set_title('Numero clienti per fascia ordine settimanale e mese')
ax.legend(title='Fasce ordine settimanale')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Filtra mesi ottobre, novembre, dicembre e solo giorni lun-ven
months_to_consider = [10, 11, 12]
filtered_q4 = delivery_data[
    (delivery_data['delivery_date'].dt.month.isin(months_to_consider)) &
    (delivery_data['delivery_date'].dt.dayofweek < 5)  # lun=0,...,ven=4
].copy()

# Ordina per data
filtered_q4 = filtered_q4.sort_values('delivery_date')

# Calcola il lunedì di ogni settimana (week_start_date)
filtered_q4['week_start_date'] = filtered_q4['delivery_date'] - pd.to_timedelta(filtered_q4['delivery_date'].dt.weekday, unit='d')

# Raggruppa per settimana (week_start_date) e cliente, conta ordini
orders_per_week = filtered_q4.groupby(['week_start_date', 'location_id']).size().reset_index(name='orders_week')

max_orders = 5

# Prendi solo ordini da 1 a 5 (escluso 0)
orders_per_week = orders_per_week[orders_per_week['orders_week'].between(1, max_orders)]

# Lista settimane ordinate
weeks = sorted(orders_per_week['week_start_date'].unique())

# Costruisci counts ordini per settimana e numero ordini
clients_per_order_count = []
for week in weeks:
    data_week = orders_per_week[orders_per_week['week_start_date'] == week]
    counts = data_week['orders_week'].value_counts()
    counts = counts.reindex(range(1, max_orders + 1), fill_value=0)
    clients_per_order_count.append(counts)

# DataFrame con i risultati
df_clients_per_order = pd.DataFrame(clients_per_order_count, index=weeks).fillna(0).astype(int)

# Funzione per creare label date da lunedì a venerdì
def make_week_label(week_start_date):
    end_date = week_start_date + pd.Timedelta(days=4)  # venerdì
    return f"{week_start_date.strftime('%d-%m')} - {end_date.strftime('%d-%m')}"

weeks_labels_corrected = [make_week_label(w) for w in weeks]
# ... codice precedente invariato fino al plot ...

highlight_offset_map = {4: 3, 5: 33}  # offset label per 4 e 5 ordini

fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(weeks))

bottom = np.zeros(len(weeks))
colors = plt.cm.tab20.colors[1:6]

labels = [f'{i} ordine{"i" if i > 1 else ""}' for i in range(1, max_orders + 1)]

for i, label in enumerate(labels, start=1):
    bars = ax.bar(x, df_clients_per_order[i], bottom=bottom, color=colors[i-1], label=label)
    for bar in bars:
        h = bar.get_height()
        if h > 0:
            label_text = str(int(h))  # eliminare .0
            if i in highlight_offset_map:
                offset = highlight_offset_map[i]
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + h + offset,
                        label_text, ha='center', va='bottom', fontsize=8, color=colors[i-1])
            else:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + h/2,
                        label_text, ha='center', va='center', fontsize=8, color='black')
    bottom += df_clients_per_order[i]

ax.set_xticks(x)
ax.set_xticklabels(weeks_labels_corrected, rotation=90)
ax.set_xlabel('Settimana lavorativa (dal lunedì al venerdì)')
ax.set_ylabel('Numero clienti')
ax.set_title('Clienti per numero preciso di ordini a settimana (ottobre, novembre, dicembre - lun-ven)')
ax.legend(title='Numero ordini settimana')

ax.set_ylim(0, df_clients_per_order.values.max() * 1.4)

plt.tight_layout()
plt.show()


In [ ]:
# Trova tutti i clienti che almeno una volta hanno fatto 4 o 5 ordini nella stessa settimana
clients_4_5_orders = orders_per_week[orders_per_week['orders_week'].isin([4, 5])]
unique_clients_4_5_orders = clients_4_5_orders['location_id'].unique()

# DataFrame con i dettagli delle settimane in cui hanno fatto 4 o 5 ordini
details_clients_4_5_orders = orders_per_week[
    (orders_per_week['location_id'].isin(unique_clients_4_5_orders)) &
    (orders_per_week['orders_week'].isin([4, 5]))
]

# Ottieni lista unica di location_id che almeno una volta hanno fatto 4 o 5 ordini
cluster_clients_4_5_orders = list(unique_clients_4_5_orders)
# Esempio se devi usarla come input per routing (una lista dentro una lista, se serve compatibile col tuo sistema):
cluster_for_routing = [cluster_clients_4_5_orders]
print("Numero clienti nel cluster 4-5 ordini:", len(cluster_clients_4_5_orders))


In [ ]:

# Notebook esterno con tutte le funzioni di routing
import import_ipynb
import performance_calc as pc

# Usa la lista cluster creata prima
# cluster_clients_4_5_orders = lista piatta di location_id

results_stats = pc.single_cluster_stats_with_cache(
    cluster_location_ids = cluster_clients_4_5_orders,
    time_limit = 7,      # oppure il valore che preferisci
    cache = None,        # se non hai una cache da passare
    verbose = True,      # se vuoi stampare output di debug
    max_workers = None   # puoi specificare se vuoi parallelismo
)

print("Risultati performance cluster clienti con 4-5 ordini:")
print(results_stats)



In [ ]:
print(results_stats)

In [ ]:
# Seleziona solo le colonne richieste ordinate per giorno della settimana
df = results_stats[0]

table_stats = df[['weekday', 'mean_minutes', 'max_overtime_minutes', 'n_overtime_days']]


print(table_stats)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
# Estrai delivery_points dai dati di consegna
# Supponendo che delivery_data abbia colonne 'location_id', 'lat', 'lon' per i punti
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)

# Filtra solo i clienti del cluster
df_cluster_points = delivery_points[delivery_points['location_id'].isin(cluster_clients_4_5_orders)]

# Centro Milano Comune
milano_lat, milano_lon = 45.4642, 9.19

plt.figure(figsize=(8,8))
plt.scatter(df_cluster_points['lon'], df_cluster_points['lat'], alpha=0.7, s=25, label='Clienti cluster')
plt.scatter(milano_lon, milano_lat, color='red', label='Centro Milano', marker='x', s=120)

# Cerchio di circa 7km di raggio intorno a Milano
cerchio = Circle((milano_lon, milano_lat), 0.065, fill=False, color='blue', lw=2, label='Milano Comune ~7km')
plt.gca().add_patch(cerchio)

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title('Clienti con 4-5 ordini a settimana')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Usa tutto il dataset delivery_data, considerando solo giorni lun-ven
filtered_all = delivery_data[
    (delivery_data['delivery_date'].dt.dayofweek < 5)  # lun=0,...,ven=4
].copy()

# Ordina per data
filtered_all = filtered_all.sort_values('delivery_date')

# Calcola il lunedì di ogni settimana (week_start_date)
filtered_all['week_start_date'] = filtered_all['delivery_date'] - pd.to_timedelta(filtered_all['delivery_date'].dt.weekday, unit='d')

# Raggruppa per settimana (week_start_date) e cliente, conta ordini
orders_per_week_all = filtered_all.groupby(['week_start_date', 'location_id']).size().reset_index(name='orders_week')

max_orders = 5

# Prendi solo ordini da 1 a 5 (escluso 0)
orders_per_week_all = orders_per_week_all[orders_per_week_all['orders_week'].between(1, max_orders)]

# Lista settimane ordinate
weeks_all = sorted(orders_per_week_all['week_start_date'].unique())

# Costruisci counts ordini per settimana e numero ordini
clients_per_order_count_all = []
for week in weeks_all:
    data_week = orders_per_week_all[orders_per_week_all['week_start_date'] == week]
    counts = data_week['orders_week'].value_counts()
    counts = counts.reindex(range(1, max_orders + 1), fill_value=0)
    clients_per_order_count_all.append(counts)

# DataFrame con i risultati
df_clients_per_order_all = pd.DataFrame(clients_per_order_count_all, index=weeks_all).fillna(0).astype(int)

# Funzione per creare label date da lunedì a venerdì
def make_week_label(week_start_date):
    end_date = week_start_date + pd.Timedelta(days=4)  # venerdì
    return f"{week_start_date.strftime('%d-%m-%Y')} - {end_date.strftime('%d-%m-%Y')}"

weeks_labels_corrected_all = [make_week_label(w) for w in weeks_all]

# Plot stacked bar chart con highlight predefinito
highlight_offset_map = {4: 3, 5: 33}  # offset label per 4 e 5 ordini

fig, ax = plt.subplots(figsize=(20,12))
x = np.arange(len(weeks_all))

bottom = np.zeros(len(weeks_all))
colors = plt.cm.tab20.colors[1:6]

labels = [f'{i} ordine{"i" if i > 1 else ""}' for i in range(1, max_orders + 1)]

for i, label in enumerate(labels, start=1):
    bars = ax.bar(x, df_clients_per_order_all[i], bottom=bottom, color=colors[i-1], label=label)
    for bar in bars:
        h = bar.get_height()
        if h > 0:
            label_text = str(int(h))  # eliminare .0
            if i in highlight_offset_map:
                offset = highlight_offset_map[i]
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + h + offset,
                        label_text, ha='center', va='bottom', fontsize=8, color=colors[i-1])
            else:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_y() + h/2,
                        label_text, ha='center', va='center', fontsize=8, color='black')
    bottom += df_clients_per_order_all[i]

ax.set_xticks(x)
ax.set_xticklabels(weeks_labels_corrected_all, rotation=90)
ax.set_xlabel('Settimana lavorativa (dal lunedì al venerdì)')
ax.set_ylabel('Numero clienti')
ax.set_title('Clienti per numero preciso di ordini a settimana (tutte le settimane - lun-ven)')
ax.legend(title='Numero ordini settimana')

ax.set_ylim(0, df_clients_per_order_all.values.max() * 1.4)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Filtra solo giorni lun-ven e calcola settimana inizio lunedì
filtered_all = delivery_data[
    (delivery_data['delivery_date'].dt.dayofweek < 5)
].copy()
filtered_all['week_start_date'] = filtered_all['delivery_date'] - pd.to_timedelta(filtered_all['delivery_date'].dt.weekday, unit='d')

# Calcola quantità e numero di ordini per location_id e settimana
week_stats = filtered_all.groupby(['week_start_date', 'location_id']).agg(
    orders_week=('delivery_date', 'count'),
    quantity_week=('quantity', 'sum')
).reset_index()

# Filtro clienti con numero ordini settimana in [3,4,5]
week_stats_filtered = week_stats[week_stats['orders_week'].isin([3, 4, 5])]

# Salva il dettaglio settimana x location_id in CSV
week_stats_filtered.to_csv('dettaglio_settimane_per_cliente.csv', index=False)

# Calcolo il numero totale di settimane nel dataset
total_weeks = filtered_all['week_start_date'].nunique()

# Conta in quante settimane ciascun cliente ha ordinato 3,4 o 5 volte
client_weeks_counts = week_stats_filtered.groupby('location_id')['week_start_date'].nunique()

# Filtra clienti con almeno 85% presenza
clients_85_percent = client_weeks_counts[client_weeks_counts >= 0.70 * total_weeks].index

# Calcola quantità totale e numero totale ordini per cliente su tutto l'anno
total_stats = delivery_data.groupby('location_id').agg(
    quantity_total=('quantity', 'sum'),
    orders_total=('delivery_date', 'count')
).reset_index()

# Filtra solo i clienti ricorrenti (>=85%)
top_clients_stats = total_stats[total_stats['location_id'].isin(clients_85_percent)]

# Salva questi dati in CSV
top_clients_stats.to_csv('clienti_top_70_percent.csv', index=False)

print("File 'dettaglio_settimane_per_cliente.csv' e 'clienti_top_70_percent.csv' salvati con successo.")


In [ ]:
# Filtra clienti che sono in fascia "3-4 ordini/settimana" o "Più di 4 ordini/settimana"
high_freq_clients = orders_count[
    orders_count['order_frequency'].isin(['3-4 ordini/settimana', 'Più di 4 ordini/settimana'])
].copy()

# Calcola quantità totale per cliente
total_quantity = filtered.groupby('location_id')['quantity'].sum().reset_index(name='total_quantity')

# Unisci quantità con clienti ad alta frequenza
high_freq_clients = high_freq_clients.merge(total_quantity, on='location_id', how='left')

# Estrai lat/lon per questi clienti senza duplicati
clients_data = delivery_data[delivery_data['location_id'].isin(high_freq_clients['location_id'])][['location_id', 'lat', 'lon']].drop_duplicates()

# Stampa tabella con id cliente, fascia, quantità totale
print(high_freq_clients[['location_id', 'order_frequency', 'total_orders', 'total_quantity']])

# Plot mappa
plt.figure(figsize=(12, 9))
plt.scatter(clients_data['lon'], clients_data['lat'], color='darkgreen', s=60, edgecolor='black', alpha=0.8)
for _, row in clients_data.iterrows():
    loc_id = str(int(row['location_id'])) if pd.notnull(row['location_id']) else ''
    plt.text(row['lon'], row['lat'], loc_id, fontsize=8, ha='right', va='bottom', color='darkgreen')

plt.title('Clienti con 3-4 o più di 4 ordini a settimana')
plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assumendo delivery_data, con le colonne anno-mese e settimana già aggiunte e filtro mesi fatto:
# 1. Filtra i clienti che ordinano 4 o più volte a settimana (fascia '4 o più ordini/settimana')
clients_4plus = avg_weekly_orders[avg_weekly_orders['order_frequency'] == '4 o più ordini/settimana']

# Lista mesi distinti ordinati
mesi = sorted(clients_4plus['year_month'].unique())

# Mappa colori (tab20)
cmap = plt.get_cmap('tab20', len(mesi))
color_map = {mese: cmap(i) for i, mese in enumerate(mesi)}

plt.figure(figsize=(14, 10))

# Per ogni mese plotta i clienti
for mese in mesi:
    cliente_mese = clients_4plus[clients_4plus['year_month'] == mese]['location_id'].unique()
    data_cliente = delivery_data[delivery_data['location_id'].isin(cliente_mese)][['location_id', 'lat', 'lon']].drop_duplicates()
    
    plt.scatter(data_cliente['lon'], data_cliente['lat'],
                color=color_map[mese], label=str(mese), s=50, edgecolor='black', alpha=0.7)
    
    for _, row in data_cliente.iterrows():
        loc_id = str(int(row['location_id'])) if pd.notnull(row['location_id']) else ''
        plt.text(row['lon'], row['lat'], loc_id, fontsize=8, ha='right', va='bottom')

plt.title('Clienti con 4 o più ordini/settimana per mese (esclusi gennaio e giugno)')
plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.legend(title='Mese', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Usa tutto il dataset o filtra se serve
data_filtered = delivery_data.copy()

# Aggiungi colonna anno-settimana
data_filtered['year_week'] = data_filtered['delivery_date'].dt.to_period('W')

# Filtra solo ordini validi
valid_orders = data_filtered[data_filtered['is_event'] == 1]

# Conta ordini per cliente e settimana
weekly_orders = valid_orders.groupby(['location_id', 'year_week']).size().reset_index(name='num_orders_per_week')

# Per ogni cliente, verifica se in TUTTE le settimane ha almeno 2 ordini
def at_least_two_per_week(df):
    return (df['num_orders_per_week'] >= 2).all()

client_all_weeks_2plus = weekly_orders.groupby('location_id').filter(at_least_two_per_week)['location_id'].unique()

print(f"Clienti che ordinano almeno 2 volte in tutte le settimane attive: {len(client_all_weeks_2plus)}")

# Estrai lat/lon per questi clienti
clients_data = delivery_data[delivery_data['location_id'].isin(client_all_weeks_2plus)][['location_id', 'lat', 'lon']].drop_duplicates()

# Plot geografico
plt.figure(figsize=(12, 9))
plt.scatter(clients_data['lon'], clients_data['lat'], color='blue', s=50, edgecolor='black', alpha=0.7)
plt.title('Clienti che ordinano ≥2 volte a settimana IN TUTTE le settimane')
plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.grid(True)
plt.show()


# 2. Set up comportamento del singolo cliente

In [ ]:
# --- 1. Parametri di base ---
qty_col = 'quantity'  # colonna quantità di pacchi

# Range temporale totale
min_date = delivery_data['delivery_date'].min()
max_date = delivery_data['delivery_date'].max()
total_weeks = max(1, (max_date - min_date).days / 7.0)

# --- 2. Aggregazioni principali per location_id ---
agg = delivery_data.groupby('location_id').agg(
    total_packs = (qty_col, 'sum'),           # totale pacchi nel periodo
    mean_packs = (qty_col, 'mean'),           # media pacchi per ordine
    std_packs = (qty_col, 'std'),             # deviazione standard pacchi
    n_orders = (qty_col, 'count'),            # numero totale ordini
    first_order = ('delivery_date', 'min'),   # data primo ordine
    last_order = ('delivery_date', 'max')     # data ultimo ordine
).reset_index()

# --- 3. Feature derivate ---
# Media settimanale di pacchi
agg['weekly_avg'] = agg['total_packs'] / total_weeks

# Coefficiente di variazione = std / media
agg['cv_packs'] = agg['std_packs'] / agg['mean_packs'].replace(0, np.nan)
agg['cv_packs'] = agg['cv_packs'].fillna(0)  # riempi NaN (clienti con un solo ordine)

# --- 4. Giorni della settimana attivi ---
# Dizionario per tradurre numero giorno -> nome
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven'}

# Trova i giorni della settimana distinti in cui il cliente ha ordinato
days_per_client = delivery_data.groupby('location_id')['weekday_num'].unique()

# Numero di giorni distinti
agg['num_weekdays_active'] = agg['location_id'].map(lambda x: len(days_per_client.get(x, [])))

# Giorni effettivi in formato stringa
agg['weekdays_active_list'] = agg['location_id'].map(
    lambda x: ', '.join([weekday_map[d] for d in days_per_client.get(x, [])])
)

df_client = agg.copy()

# --- 5. Ordini per mese (colonne orders_m1 ... orders_m12) ---
delivery_data['month'] = delivery_data['delivery_date'].dt.month

monthly_counts = (
    delivery_data
    .groupby(['location_id', 'month'])[qty_col]
    .count()
    .unstack(fill_value=0)
)

# Rinomina colonne
monthly_counts.columns = [f'orders_m{int(m)}' for m in monthly_counts.columns]

# Unisci al df_client
df_client = df_client.merge(monthly_counts, on='location_id', how='left')

# --- 6. Ordini per giorno della settimana (colonne mon_orders ... sun_orders) ---
delivery_data['weekday'] = delivery_data['delivery_date'].dt.dayofweek  # 0=Lun ... 6=Dom

weekday_counts = (
    delivery_data
    .groupby(['location_id', 'weekday'])[qty_col]
    .count()
    .unstack(fill_value=0)
)

weekday_counts.columns = [
    'mon_orders', 'tue_orders', 'wed_orders', 'thu_orders',
    'fri_orders'
]

# Unisci al df_client
df_client = df_client.merge(weekday_counts, on='location_id', how='left')

# --- 8. Check finale ---
print("Colonne finali:", df_client.columns.tolist())
print(df_client.head(10))

# ===================================================================
# Significato delle colonne di df_client
# ===================================================================
# location_id           : ID univoco del cliente / luogo di consegna
# total_packs           : Totale pacchi ordinati dal cliente nel periodo osservato
# mean_packs            : Quantità media di pacchi per ordine
# std_packs             : Deviazione standard della quantità dei pacchi (quanto variano gli ordini)
# n_orders              : Numero totale di ordini effettuati dal cliente
# first_order           : Data del primo ordine registrato
# last_order            : Data dell'ultimo ordine registrato
# weekly_avg            : Media settimanale di pacchi ordinati (total_packs / numero settimane totali)
# cv_packs              : Coefficiente di variazione della quantità di pacchi per ordine (std_packs / mean_packs)
# num_weekdays_active   : Numero di giorni della settimana distinti in cui il cliente ha ordinato
# weekdays_active_list  : Lista dei giorni della settimana in cui il cliente ha ordinato (es. 'Lun, Mer, Ven')
# orders_m1             : Numero di ordini effettuati a gennaio
# orders_m6             : Numero di ordini effettuati a giugno
# orders_m7             : Numero di ordini effettuati a luglio
# orders_m8             : Numero di ordini effettuati ad agosto
# orders_m9             : Numero di ordini effettuati a settembre
# orders_m10            : Numero di ordini effettuati a ottobre
# orders_m11            : Numero di ordini effettuati a novembre
# orders_m12            : Numero di ordini effettuati a dicembre
# mon_orders            : Numero di ordini effettuati di lunedì
# tue_orders            : Numero di ordini effettuati di martedì
# wed_orders            : Numero di ordini effettuati di mercoledì
# thu_orders            : Numero di ordini effettuati di giovedì
# fri_orders            : Numero di ordini effettuati di venerdì
# sat_orders            : Numero di ordini effettuati di sabato
# sun_orders            : Numero di ordini effettuati di domenica
# ===================================================================



# 3. Analisi sulla domanda e sui clienti

## Analisi per fascia di quantità

In [ ]:
# Questo blocco filtra e mostra i clienti più "top" in termini di quantità totale ordinata (>8000 pacchi),
# ordinandoli e visualizzando alcune metriche chiave come numero di ordini, media settimanale, coefficiente di variazione
# e giorni della settimana in cui effettuano gli ordini.

# Filtra i clienti con totale pacchi > 700
top_clients_8k = df_client[df_client['total_packs'] > 7000]  # prende solo clienti con più di 8000 pacchi

# Ordina in ordine decrescente per total_packs
top_clients_8k = top_clients_8k.sort_values(by='total_packs', ascending=False)  # ordina dal più "pesante" al più leggero

# Mostra le colonne principali
print("Clienti con quantità totale > 8000:")
print(top_clients_8k[['location_id', 'total_packs', 'n_orders', 'weekly_avg', 'cv_packs', 'num_weekdays_active', 'weekdays_active_list']])
# mostra ID cliente, pacchi totali, numero di ordini, media settimanale, coefficiente di variazione dei pacchi,
# numero di giorni della settimana attivi e lista dei giorni della settimana


In [ ]:
# --- Creazione delle fasce di domanda e grafico ---
# Questo blocco suddivide i clienti in fasce di quantità totale di pacchi ordinati,
# conta quanti clienti rientrano in ogni fascia, visualizza il tutto con un grafico a barre
# e stampa le informazioni principali di ciascun cliente nelle fasce.

# Definisci i bordi degli intervalli
bins = [0, 500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, df_client['total_packs'].max()]  # intervalli di domanda
labels = [
    '0-500', '500-1000', '1000-2000', '2000-3000', '3000-4000',
    '4000-5000', '5000-6000', '6000-7000', '7000-8000', '8000+'  # etichette per le fasce
]

# Crea una colonna con la fascia di domanda
df_client['demand_range'] = pd.cut(df_client['total_packs'], bins=bins, labels=labels, right=False)  # assegna fascia a ciascun cliente

# Conta quanti clienti ci sono per fascia
demand_counts = df_client['demand_range'].value_counts().sort_index()  # conta clienti per fascia ordinata
print("\nNumero di clienti per fascia di domanda:")
print(demand_counts)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


# Supponendo che bins, labels e df_client['demand_range'] siano già definiti come nel tuo esempio


# Somma quantità totale ordinata per fascia
quantity_per_fascia = df_client.groupby('demand_range')['total_packs'].sum().sort_index()


# Conta quanti clienti per fascia
demand_counts = df_client['demand_range'].value_counts().sort_index()


print("\nNumero di clienti per fascia di domanda:")
print(demand_counts)


print("\nQuantità totale ordinata per fascia di domanda:")
print(quantity_per_fascia)


# Grafico
plt.figure(figsize=(12,6))
bars = plt.bar(demand_counts.index, demand_counts.values, color='skyblue', edgecolor='black')
pop_per_fascia = demand_counts

for bar, pop, qty in zip(bars, pop_per_fascia, quantity_per_fascia):
    height = bar.get_height()
    label_y = height + 0.5  # posizione dell'etichetta numero clienti
    
    # Etichetta numero clienti sopra barra
    plt.text(bar.get_x() + bar.get_width()/2, label_y, str(int(height)), ha='center', va='bottom', fontsize=10)
    
    # Quantità totale (Q:) posizionata in base all'altezza della barra e distanza dall'etichetta
    if height > 60:
       plt.text(bar.get_x() + bar.get_width()/2, label_y - 15, f"Q: {int(qty)}",
                 ha='center', va='top', fontsize=7, color='darkblue', fontweight='bold')
    else:
        plt.text(bar.get_x() + bar.get_width()/2, label_y + 70, f"Q: {int(qty)}",
                 ha='center', va='bottom', fontsize=7, color='darkblue', fontweight='bold')

plt.xlabel("Fasce di domanda (colli totali)")
plt.ylabel("Numero di clienti")
plt.title("Distribuzione dei clienti per quantità totale di colli ordinati")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Supponiamo df_client già definito con colonne: total_packs, n_orders, demand_range

# Aggiungi la quantità media per ordine
df_client['avg_packs_per_order'] = df_client['total_packs'] / df_client['n_orders']

# Calcola media, varianza, std e CV per fascia
agg_stats = df_client.groupby('demand_range')['avg_packs_per_order'].agg(['mean', 'var', 'std']).round(2)
agg_stats['cv'] = (agg_stats['std'] / agg_stats['mean']).round(2)

# Numero di clienti per fascia
pop_per_fascia = df_client.groupby('demand_range').size()

# Grafico
plt.figure(figsize=(12,6))
bars = plt.bar(agg_stats.index, agg_stats['mean'], color='orange', edgecolor='black')

for i, (bar, pop) in enumerate(zip(bars, pop_per_fascia)):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f"{height:.2f}", ha='center', va='bottom', fontsize=10)
    plt.text(bar.get_x() + bar.get_width()/2, 6.5, f"n={pop}", ha='center', va='top', fontsize=9, color='black')

    # Inserisci varianza, std e CV dentro la barra in piccolo e blu
    var_val = agg_stats.iloc[i]['var']
    std_val = agg_stats.iloc[i]['std']
    cv_val = agg_stats.iloc[i]['cv']
    plt.text(bar.get_x() + bar.get_width()/2, height / 2 + 1,
             f"Var: {var_val}\nSD: {std_val}\nCV: {cv_val}",
             ha='center', va='center', fontsize=7, color='blue', fontweight='bold')

plt.xlabel("Fasce di domanda (pacchi totali)")
plt.ylabel("Quantità media per ordine")
plt.title("Quantità media per ordine con varianza, SD e CV per fascia di domanda")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aggiungi la quantità media per ordine, se non presente
if 'avg_packs_per_order' not in df_client.columns:
    df_client['avg_packs_per_order'] = df_client['total_packs'] / df_client['n_orders']

# Calcola varianza, deviazione standard e CV per fascia
agg_stats = df_client.groupby('demand_range')['avg_packs_per_order'].agg(['var', 'std', 'mean']).round(2)
agg_stats['cv'] = (agg_stats['std'] / agg_stats['mean']).round(2)

# Crea il boxplot
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='demand_range', y='avg_packs_per_order', data=df_client, color='orange')

# Inserisci le statistiche sopra ogni boxplot
for i, label in enumerate(ax.get_xticklabels()):
    fascia_label = label.get_text()
    if fascia_label in agg_stats.index:
        stats = agg_stats.loc[fascia_label]
        ax.text(i, df_client['avg_packs_per_order'].max() * 1.15,
                f"Var: {stats['var']}\nSD: {stats['std']}\nCV: {stats['cv']}",
                ha='center', va='bottom', fontsize=10, color='blue')

plt.xlabel("Fasce di domanda (pacchi totali)")
plt.ylabel("Quantità media per ordine")
plt.title("Distribuzione quantità media per ordine con varianza, SD e CV")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Filtra righe con is_event = 1
filtered_data = delivery_data[delivery_data['is_event'] == 1]

# Raggruppa per location_id e somma quantity
quantity_per_location = filtered_data.groupby('location_id')['quantity'].sum().reset_index()

print(quantity_per_location.head())


## Analisi per fascia di ordini

In [ ]:
# Raggruppa per fascia e calcola statistiche
stats_per_fascia = df_client.groupby('demand_range')['mean_packs'].agg(['mean', 'std', 'var']).round(2)

# Calcola coefficiente di variazione
stats_per_fascia['cv'] = (stats_per_fascia['std'] / stats_per_fascia['mean']).round(2)

print("Statistiche per fascia di domanda:")
print(stats_per_fascia)


"""
Significato delle statistiche:
- mean (media): la media della quantità media per ordine all’interno di ogni fascia
- std (deviazione standard): misura quanto i valori di quantità media per ordine si discostano dalla media;
  più alto indica più dispersione o variazione
- var (varianza): è la media dei quadrati delle differenze dei valori dalla media, misura più "numerica" della dispersione
- cv (coefficiente di variazione): std diviso mean, misura la variazione relativa rispetto alla media;
  un valore basso indica che i dati sono strettamente raggruppati intorno alla media,
  un valore alto indica che la media ha alta variabilità all’interno della fascia

Come interpretare la variazione della media all’interno di ogni fascia:
- Se cv < 0.25: la media è abbastanza stabile, i valori sono vicini tra loro
- Se cv tra 0.25 e 0.5: variabilità moderata
- Se cv > 0.5: la media varia molto, i valori sono molto dispersi

Quindi il coefficiente di variazione (cv) è il miglior indicatore per capire quanto varia la media nel gruppo.
"""




In [ ]:
import matplotlib.pyplot as plt

# Somma quantità totale ordinata per fascia di numero di ordini
quantity_per_orders_range = df_client.groupby('orders_range')['total_packs'].sum().sort_index()

# Conta quanti clienti per fascia (gia' fatto)
orders_counts = df_client['orders_range'].value_counts().sort_index()

print("Numero di clienti per fascia di numero di ordini:")
print(orders_counts)

print("\nQuantità totale ordinata per fascia di numero di ordini:")
print(quantity_per_orders_range)

# Grafico
plt.figure(figsize=(14,6))
bars = plt.bar(orders_counts.index, orders_counts.values, color='lightcoral', edgecolor='black')

# Annotazioni: numero clienti sopra barra e quantità totale con posizionamento a seconda dell'altezza
for bar, count, qty in zip(bars, orders_counts, quantity_per_orders_range):
    height = bar.get_height()
    label_y = height + 2

    # Numero clienti sopra la barra
    plt.text(bar.get_x() + bar.get_width()/2, label_y, str(int(count)),
             ha='center', va='bottom', fontsize=10)

    # Quantità totale posizionata in base all’altezza della barra e distanza dall’etichetta
    if height > 60:
        plt.text(bar.get_x() + bar.get_width()/2, label_y - 15, f"Q: {int(qty)}",
                 ha='center', va='top', fontsize=6, color='darkred', fontweight='bold')
    else:
        plt.text(bar.get_x() + bar.get_width()/2, label_y + 40, f"Q: {int(qty)}",
                 ha='center', va='bottom', fontsize=6, color='darkred', fontweight='bold')

plt.xlabel("Fasce di numero di ordini per cliente")
plt.ylabel("Numero di clienti")
plt.title("Distribuzione dei clienti per numero di ordini totali")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Filtra i clienti con almeno 75 ordini
df_high_orders = df_client[df_client['n_orders'] >= 75].copy()

# Seleziona solo le colonne rilevanti
df_summary = df_high_orders[['location_id', 'n_orders', 'total_packs']]

# Ordina decrescente per numero di ordini
df_summary = df_summary.sort_values(by='n_orders', ascending=False)

# Stampa la tabella
print(df_summary.to_string(index=False) )

from tabulate import tabulate

# Converti il dataframe in tabella
table = tabulate(df_summary, headers='keys', tablefmt='pretty', showindex=False)
print(table)



In [ ]:
import matplotlib.pyplot as plt

# Ordina i clienti in base alla quantità totale ordinata
df_summary = df_summary.sort_values(by='total_packs', ascending=False)

# Crea la figura e i due assi
fig, ax1 = plt.subplots(figsize=(14,7))
ax2 = ax1.twinx()  # secondo asse Y per i punti (numero di ordini)

# --- Barre: Totale pacchi ordinati ---
bars = ax1.bar(
    df_summary['location_id'].astype(str),
    df_summary['total_packs'],
    color='lightcoral',
    edgecolor='black',
    alpha=0.85,
    label='Totale pacchi ordinati'
)

# --- Punti: Numero di ordini (senza linea) ---
ax2.scatter(
    df_summary['location_id'].astype(str),
    df_summary['n_orders'],
    color='blue',
    s=80,  # dimensione punti
    label='Numero ordini',
    zorder=5  # sopra le barre
)

# --- Etichette sopra le barre ---
for i, row in enumerate(df_summary.itertuples()):
    ax1.text(
        i,
        row.total_packs + (row.total_packs * 0.02),  # piccolo margine sopra la barra
        str(row.total_packs),
        ha='center',
        va='bottom',
        fontsize=9,
        color='black'
    )

# --- Configurazioni asse sinistro (Totale pacchi) ---
ax1.set_xlabel("Cliente (location_id)", fontsize=12)
ax1.set_ylabel("Totale pacchi ordinati", fontsize=12, color='lightcoral')
ax1.tick_params(axis='y', labelcolor='lightcoral')

# --- Configurazioni asse destro (Numero ordini) ---
ax2.set_ylabel("Numero ordini", fontsize=12, color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

# --- Titolo e griglia ---
plt.title("Totale pacchi ordinati e numero di ordini per cliente (≥75 ordini)", fontsize=14, fontweight='bold')
ax1.set_xticks(range(len(df_summary['location_id'])))
ax1.set_xticklabels(df_summary['location_id'].astype(str), rotation=45, ha='right')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# --- Legenda ---
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right')

plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Calcola la media di pacchi per ordine per ogni cliente
df_summary['avg_packs_per_order'] = df_summary['total_packs'] / df_summary['n_orders']

# Ordina i clienti in base alla media calcolata
df_summary = df_summary.sort_values(by='n_orders', ascending=False)

# --- Grafico a barre ---
plt.figure(figsize=(14,7))

bars = plt.bar(
    df_summary['location_id'].astype(str),
    df_summary['avg_packs_per_order'],
    color='lightseagreen',
    edgecolor='black',
    alpha=0.85
)

# --- Aggiungi etichette sopra le barre ---
for i, row in enumerate(df_summary.itertuples()):
    plt.text(
        i,
        row.avg_packs_per_order + 0.2,  # piccolo margine sopra la barra
        f"{row.avg_packs_per_order:.2f}",  # valore arrotondato a 2 decimali
        ha='center',
        va='bottom',
        fontsize=9,
        color='black'
    )

# --- Configurazioni grafiche ---
plt.xlabel("Cliente (location_id)", fontsize=12)
plt.ylabel("Media pacchi per ordine", fontsize=12)
plt.title("Media pacchi per ordine per cliente (≥75 ordini)", fontsize=14, fontweight='bold')

# Ruota le etichette sull'asse X per leggibilità
plt.xticks(rotation=45, ha='right')

# Aggiunge griglia sull'asse Y
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14,6))
plt.scatter(range(len(df_high_orders)), df_high_orders['n_orders'], color='red')
plt.plot(range(len(df_high_orders)), df_high_orders['n_orders'], linestyle='--', color='blue', alpha=0.5)
plt.xticks(range(len(df_high_orders)), df_high_orders['location_id'], rotation=45)
plt.xlabel("Cliente")
plt.ylabel("Numero di ordini")
plt.title("Clienti con almeno 70 ordini")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# --- 1. Calcolo della media pacchi per ordine per cliente ---
df_client['avg_packs_per_order'] = df_client['total_packs'] / df_client['n_orders']

# --- 2. Calcolo della media per fascia ---
avg_packs_per_order_by_range = (
    df_client.groupby('orders_range')['avg_packs_per_order']
    .mean()
    .reindex(labels_orders, fill_value=0)  # mantiene l'ordine corretto delle fasce
)

# --- 3. Numero di clienti per fascia (utile come riferimento) ---
count_clients_per_range = (
    df_client.groupby('orders_range')['location_id']
    .count()
    .reindex(labels_orders, fill_value=0)
)

# --- 4. Calcolo varianza per fascia ---
varianza_per_range = (
    df_client.groupby('orders_range')['avg_packs_per_order']
    .var()
    .reindex(labels_orders, fill_value=0)
)

# --- 5. Calcolo coefficiente di variazione (cv) per fascia ---
std_per_range = (
    df_client.groupby('orders_range')['avg_packs_per_order']
    .std()
    .reindex(labels_orders, fill_value=0)
)
cv_per_range = (std_per_range / avg_packs_per_order_by_range).fillna(0)

# --- 6. Calcolo quantità totale per fascia ---
quantity_per_range = (
    df_client.groupby('orders_range')['total_packs']
    .sum()
    .reindex(labels_orders, fill_value=0)
)

# Stampa risultati
print("\nMedia pacchi per ordine per fascia di numero di ordini:")
print(avg_packs_per_order_by_range)

print("\nNumero di clienti per fascia:")
print(count_clients_per_range)

print("\nVarianza per fascia:")
print(varianza_per_range)

print("\nCoefficiente di variazione per fascia:")
print(cv_per_range)

print("\nQuantità totale per fascia:")
print(quantity_per_range)


# --- 7. Grafico comparativo ---
plt.figure(figsize=(14,6))
bars = plt.bar(avg_packs_per_order_by_range.index, avg_packs_per_order_by_range.values, 
               color='cornflowerblue', edgecolor='black')

for bar, var, cv, qty in zip(bars, varianza_per_range, cv_per_range, quantity_per_range):
    height = bar.get_height()
    label_y = height + 0.5
    
    # Media (rimane sempre sopra la barra)
    plt.text(bar.get_x() + bar.get_width()/2, label_y, f"{height:.2f}", ha='center', va='bottom', fontsize=10)
    
    if height > 60:
        height = - 6 # posizione base sotto la barra
        plt.text(bar.get_x() + bar.get_width()/2, base_y - 10, f"Q: {int(qty)}", ha='center', va='top', fontsize=5, color='darkblue', fontweight='bold')
        plt.text(bar.get_x() + bar.get_width()/2, base_y - 28, f"Var: {var:.2f}", ha='center', va='top', fontsize=5, color='black')
        plt.text(bar.get_x() + bar.get_width()/2, base_y - 38, f"CV: {cv:.2f}", ha='center', va='top', fontsize=6, color='black')
    else:
        base_y = label_y + 6  # posizione base sopra la barra e sopra la media
        plt.text(bar.get_x() + bar.get_width()/2, base_y + 15, f"Q: {int(qty)}", ha='center', va='bottom', fontsize=5, color='darkblue', fontweight='bold')
        plt.text(bar.get_x() + bar.get_width()/2, base_y + 28, f"Var: {var:.2f}", ha='center', va='bottom', fontsize=5, color='black')
        plt.text(bar.get_x() + bar.get_width()/2, base_y + 38, f"CV: {cv:.2f}", ha='center', va='bottom', fontsize=6, color='black')

plt.xlabel("Fasce di numero di ordini per cliente")
plt.ylabel("Media pacchi per ordine")
plt.title("Media pacchi per ordine, quantita', varianza e coefficiente di variazione per fascia")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Soglia per cv alto
cv_threshold = 0.5

# Filtra le fasce con cv alto
high_cv_mask = cv_per_range > cv_threshold
high_cv_fasce = avg_packs_per_order_by_range[high_cv_mask]
high_cv_var = varianza_per_range[high_cv_mask]
high_cv_cv = cv_per_range[high_cv_mask]
high_cv_qty = quantity_per_range[high_cv_mask]

plt.figure(figsize=(12,6))
bars = plt.bar(high_cv_fasce.index, high_cv_fasce.values, color='tomato', edgecolor='black')

for bar, var, cv, qty in zip(bars, high_cv_var, high_cv_cv, high_cv_qty):
    height = bar.get_height()
    label_y = height + 0.5

    # Media sempre sopra la barra
    plt.text(bar.get_x() + bar.get_width()/2, label_y, f"{height:.2f}", ha='center', va='bottom', fontsize=10)

        # Testo sotto o sopra a seconda dell'altezza (come nel codice precedente)
    base_y = height - 6
    plt.text(bar.get_x() + bar.get_width()/2, base_y, f"Q: {int(qty)}", ha='center', va='top', fontsize=7, color='darkred', fontweight='bold')
    plt.text(bar.get_x() + bar.get_width()/2, base_y - 25, f"Var: {var:.2f}", ha='center', va='top', fontsize=7, color='black')
    plt.text(bar.get_x() + bar.get_width()/2, base_y - 30, f"CV: {cv:.2f}", ha='center', va='top', fontsize=7, color='black')

plt.xlabel("Fasce di numero di ordini per cliente con alto CV (>0.5)")
plt.ylabel("Media pacchi per ordine")
plt.title("Fasce con coefficiente di variazione alto")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(14,6))
ax = sns.boxplot(x='orders_range', y='avg_packs_per_order', data=df_client, color='cornflowerblue')

plt.xlabel("Fasce di numero di ordini per cliente")
plt.ylabel("Quantità media per ordine")
plt.title("Distribuzione quantità media per ordine per fascia di numero di ordini")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# --- 1. Filtra i clienti con total_packs > 8000 ---
top_demand_clients = df_client[df_client['total_packs'] > 8000].copy()

# Se non ci sono clienti che soddisfano la condizione
if top_demand_clients.empty:
    print("Nessun cliente ha ordinato più di 8000 pacchi.")
else:
    # --- 2. Calcolo degli anni attivi per ogni cliente ---
    top_demand_clients['years_active'] = (
        (top_demand_clients['last_order'] - top_demand_clients['first_order']).dt.days / 365.25
    ).apply(lambda x: max(1, round(x, 2)))  # minimo 1 anno per evitare divisione per 0

    # --- 3. Calcolo ordini medi annuali ---
    top_demand_clients['avg_orders_per_year'] = (
        top_demand_clients['n_orders'] / top_demand_clients['years_active']
    ).round(2)

    # --- 4. Calcolo pacchi medi per ordine ---
    top_demand_clients['avg_packs_per_order'] = (
        top_demand_clients['total_packs'] / top_demand_clients['n_orders']
    ).round(2)

    # --- 5. Ordina per total_packs decrescente ---
    top_demand_clients = top_demand_clients.sort_values(by='total_packs', ascending=False)

    # --- 6. Tabella finale ---
    result = top_demand_clients[['location_id', 'total_packs', 'n_orders', 
                                 'years_active', 'avg_orders_per_year', 'avg_packs_per_order']]

    print("\nClienti con più di 8000 pacchi ordinati e i loro dettagli annuali:")
    print(result)

    # --- 7. Statistiche complessive ---
    num_clients = len(result)
    total_packs_top = result['total_packs'].sum()
    avg_orders_year_total = result['avg_orders_per_year'].mean()
    avg_packs_per_order_total = result['avg_packs_per_order'].mean()

    print(f"\nNumero di clienti con più di 8000 pacchi: {num_clients}")
    print(f"Domanda totale di questi clienti: {total_packs_top}")
    print(f"Media complessiva degli ordini annuali: {avg_orders_year_total:.2f}")
    print(f"Media complessiva dei pacchi per ordine: {avg_packs_per_order_total:.2f}")

    # --- 8. Grafico: pacchi totali per cliente ---
    plt.figure(figsize=(12,6))
    plt.bar(result['location_id'].astype(str), result['total_packs'], 
            color='orange', edgecolor='black')

    # Aggiungi valori sopra le barre
    for i, val in enumerate(result['total_packs']):
        plt.text(i, val + 50, str(int(val)), ha='center', va='bottom', fontsize=9)

    plt.xlabel("Clienti (location_id)")
    plt.ylabel("Totale pacchi ordinati")
    plt.title("Clienti con più di 8000 pacchi ordinati")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # --- 9. Grafico: pacchi medi per ordine ---
    plt.figure(figsize=(12,6))
    plt.bar(result['location_id'].astype(str), result['avg_packs_per_order'], 
            color='skyblue', edgecolor='black')

    # Aggiungi valori sopra le barre
    for i, val in enumerate(result['avg_packs_per_order']):
        plt.text(i, val + 0.2, f"{val:.2f}", ha='center', va='bottom', fontsize=9)

    plt.xlabel("Clienti (location_id)")
    plt.ylabel("Pacchi medi per ordine")
    plt.title("Pacchi medi per ordine dei clienti con più di 8000 pacchi")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
# --- 1. Domanda media per cliente per mese ---

# Creiamo la colonna per anno-mese
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# Raggruppiamo per cliente e mese calcolando la media della quantità
monthly_quantity_avg = (
    delivery_data
    .groupby(['location_id', 'year_month'])['quantity']
    .mean()
    .reset_index()
)

# Pivot per avere i mesi come colonne
monthly_quantity_pivot = monthly_quantity_avg.pivot(
    index='location_id',
    columns='year_month',
    values='quantity'
)

# Stampa le prime 10 righe
print("Domanda media per cliente per mese:")
print(monthly_quantity_pivot.head(10))

# --- 2. Numero medio di ordini per cliente per mese ---

# Creiamo la colonna anno-mese (se non già creata)
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# Raggruppiamo per cliente e mese contando gli ordini
monthly_orders_count = (
    delivery_data
    .groupby(['location_id', 'year_month'])['quantity']
    .count()
    .reset_index(name='n_orders')
)

# Pivot per avere i mesi come colonne
monthly_orders_pivot = monthly_orders_count.pivot(
    index='location_id',
    columns='year_month',
    values='n_orders'
)

# Stampa le prime 10 righe
print("Numero totale di ordini per cliente per mese:")
print(monthly_orders_pivot.head(10))


In [ ]:
# Domanda media per singolo cliente per giorno della settimana
#filtro solo per is_event = True
delivery_data = delivery_data[delivery_data['is_event']]
# Crea una colonna con il giorno della settimana (0=Lunedì, 6=Domenica)
delivery_data['day_of_week'] = delivery_data['delivery_date'].dt.day_name()
# GroupBy con pivot per ottenere i giorni come colonne
weekly_avg = delivery_data.groupby(['location_id', 'day_of_week'])['quantity'].mean().reset_index()
# Pivot per avere i giorni come colonne
weekly_avg_pivot = weekly_avg.pivot(index='location_id', columns='day_of_week', values='quantity')
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekly_avg_pivot = weekly_avg_pivot[days_order]
print(weekly_avg_pivot.head(10))  # prime 10 location_id

# --- 2. Numero medio di ordini per singolo cliente per giorno della settimana ---

# Ogni riga di delivery_data rappresenta un ordine, quindi possiamo usare .count()

# Raggruppa per cliente e giorno della settimana contando gli ordini
daily_orders_count = (
    delivery_data
    .groupby(['location_id', 'day_of_week'])['quantity']
    .count()
    .reset_index(name='n_orders')
)

# Pivot per avere i giorni come colonne
daily_orders_pivot = daily_orders_count.pivot(
    index='location_id',
    columns='day_of_week',
    values='n_orders'
)

# Ordina i giorni correttamente
daily_orders_pivot = daily_orders_pivot[days_order]

# Stampa le prime righe
print("Numero totale di ordini per cliente per giorno della settimana:")
print(daily_orders_pivot.head(10))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Calcola quantità totale ordinata per cliente (location_id)
total_per_client = delivery_data.groupby('location_id')['quantity'].sum().reset_index()

# Step 2: Seleziona top 20 clienti per quantità totale
top20_clients = total_per_client.nlargest(20, 'quantity')

# Step 3: Filtra solo gli ordini per questi top 20 clienti
top20_orders = delivery_data[delivery_data['location_id'].isin(top20_clients['location_id'])]

# Step 4: Calcola media, varianza e deviazione standard per i singoli ordini di ogni cliente
stats_top20 = top20_orders.groupby('location_id')['quantity'] \
    .agg(['mean', 'var', 'std', 'count']).reset_index()

# Rinominare count in n_orders per chiarezza
stats_top20.rename(columns={'count': 'n_orders'}, inplace=True)

# Step 5: Calcola coefficiente di variazione cv = std / mean
stats_top20['cv'] = stats_top20['std'] / stats_top20['mean']

# Step 6: Aggiungi la quantità totale dal passo 1 (total_per_client)
stats_top20 = stats_top20.merge(top20_clients, on='location_id', how='left')
stats_top20.rename(columns={'quantity': 'total_packs'}, inplace=True)

# Step 7: Ordina per quantità totale decrescente
stats_top20 = stats_top20.sort_values('total_packs', ascending=False)

print(stats_top20)

# Step 8: Plot con annotazioni media, varianza e CV
plt.figure(figsize=(15, 7))
bars = plt.bar(stats_top20['location_id'].astype(str), stats_top20['mean'], color='cornflowerblue', edgecolor='black')

for i, (mean, var, cv, total) in enumerate(zip(stats_top20['mean'], stats_top20['var'], stats_top20['cv'], stats_top20['total_packs'])):
    plt.text(i, mean + 10, f"{mean:.2f}", ha='center', va='bottom', fontsize=9)
    plt.text(i, mean - 38, f"Var: {var:.2f}", ha='center', va='bottom', fontsize=7)
    plt.text(i, mean - 53, f"CV: {cv:.2f}", ha='center', va='bottom', fontsize=7)
    plt.text(i, mean + 50, f"Tot: {int(total)}", ha='center', va='bottom', fontsize=7, color='darkred')

plt.xlabel("Top 20 clienti per quantità totale")
plt.ylabel("Media colli per ordine")
plt.title("Media, varianza e coefficiente di variazione degli ordini per top 20 clienti")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# 4. Studio della domanda aggregata

In [ ]:
# Conta i giorni distinti per ogni giorno della settimana
days_per_weekday = delivery_data.groupby('weekday_name')['delivery_day'].nunique()
# Riordina per giorno da Lun a Ven (visto che hai escluso Sab e Dom)
days_per_weekday = days_per_weekday.reindex(['Lun','Mar','Mer','Gio','Ven'])
# Stampa il risultato
print("Numero di giorni presenti per ciascun giorno della settimana:")
print(days_per_weekday)

# Data minima e massima nel dataset
min_date = delivery_data['delivery_date'].min()
max_date = delivery_data['delivery_date'].max()
print("Data iniziale dei dati:", min_date)
print("Data finale dei dati:", max_date)

#QUESTO PER DIRE CHE I DATI LI ABBAMO COMPLETI: ci sono tutte le 
# settimane complete dall'inizio dei dati alla fine ma in realtà il 
# dato mancante del venerdì è perché un venerdì non ci sono ordini

In [ ]:
# --- Aggregazione quantità per giorno della settimana ---
# Raggruppa la quantità totale ordinata per giorno della settimana
# reindex per ordinare i giorni correttamente: Lun, Mar, ..., Dom
weekday_demand = delivery_data.groupby('weekday_name')['quantity'].sum().reindex(['Lun','Mar','Mer','Gio','Ven'])

# --- Creazione del grafico a barre ---
plt.figure(figsize=(10,5))  # imposta le dimensioni del grafico
bars = plt.bar(weekday_demand.index, weekday_demand.values, color='skyblue', edgecolor='black')  # barre blu con bordo nero

# Aggiungi i valori sopra ogni barra
for bar in bars:
    height = bar.get_height()  # altezza della barra = quantità totale ordinata
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, str(int(height)),  # posiziona il valore sopra la barra
             ha='center', va='bottom', fontsize=10)

# Imposta etichette e titolo
plt.xlabel("Giorno della settimana")  # etichetta asse X
plt.ylabel("Quantità totale ordinata")  # etichetta asse Y
plt.title("Domanda aggregata per giorno della settimana")  # titolo del grafico

# Mostra il grafico
plt.show()


In [ ]:
# --- Aggregazione quantità per mese ---
# Raggruppa la quantità totale ordinata per periodo (anno-mese)
monthly_demand = delivery_data.groupby('year_month')['quantity'].sum()

# --- Creazione del grafico lineare ---
plt.figure(figsize=(12,5))  # imposta le dimensioni del grafico
plt.plot(monthly_demand.index.astype(str), monthly_demand.values, marker='o', color='skyblue', linewidth=2)  # linee con marker

# Aggiungi i valori sopra ogni punto della linea
for i, value in enumerate(monthly_demand.values):
    plt.text(i, value + 0.5, str(int(value)), ha='center', va='bottom', fontsize=9)

# Imposta etichette e titolo
plt.xlabel("Mese")  # etichetta asse X
plt.ylabel("Quantità totale ordinata")  # etichetta asse Y
plt.title("Domanda aggregata mensile")  # titolo del grafico
plt.grid(True)  # mostra la griglia
plt.xticks(rotation=45)  # ruota le etichette dei mesi per leggibilità

# Mostra il grafico
plt.show()

#PARTE 2 : SE VOGLIO ELIMINARE QUELLA SOPRA BISOGNA TENERE QUESTO QUI
# --- Calcola la quantità totale per mese ---
monthly_total = delivery_data.groupby('year_month')['quantity'].sum()

# --- Conta i giorni effettivi presenti in ogni mese ---
days_per_month = delivery_data.groupby('year_month')['delivery_day'].nunique()

# --- Calcola la media giornaliera ---
monthly_avg_per_day = monthly_total / days_per_month

# --- Creazione del grafico lineare ---
plt.figure(figsize=(12,5))
plt.plot(monthly_avg_per_day.index.astype(str), monthly_avg_per_day.values, marker='o', color='skyblue', linewidth=2)

# Aggiungi i valori sopra ogni punto della linea
for i, value in enumerate(monthly_avg_per_day.values):
    plt.text(i, value + 0.5, f"{value:.1f}", ha='center', va='bottom', fontsize=9)

# Imposta etichette e titolo
plt.xlabel("Mese")
plt.ylabel("Media quantità ordinata per giorno")
plt.title("Media giornaliera della domanda per mese")
plt.grid(True)
plt.xticks(rotation=45)

# Mostra il grafico
plt.show()



In [ ]:
# --- Aggregazione quantità per settimana ---
# Raggruppa la quantità totale ordinata per settimana (resample settimanale)
weekly_demand = delivery_data.resample('W', on='delivery_date')['quantity'].sum()

# --- Creazione del grafico lineare settimanale ---
plt.figure(figsize=(12,5))  # imposta le dimensioni del grafico
plt.plot(weekly_demand.index, weekly_demand.values, marker='o', color='salmon', linewidth=2)  # linee con marker

# Aggiungi i valori sopra ogni punto della linea
for i, value in enumerate(weekly_demand.values):
    plt.text(weekly_demand.index[i], value + 0.5, str(int(value)), ha='center', va='bottom', fontsize=8)

# Imposta etichette e titolo
plt.xlabel("Settimana")  # etichetta asse X
plt.ylabel("Quantità totale ordinata")  # etichetta asse Y
plt.title("Domanda aggregata settimanale")  # titolo del grafico
plt.grid(True)  # mostra la griglia
plt.xticks(rotation=45)  # ruota le etichette delle settimane per leggibilità

# Mostra il grafico
plt.show()

#CODICE 2

weekly_days = delivery_data.resample('W', on='delivery_date')['delivery_date'].nunique()
# Calcola la media giornaliera per settimana
weekly_avg_per_day = weekly_demand / weekly_days

# --- Grafico della media settimanale ---
plt.figure(figsize=(12,5))
plt.plot(weekly_avg_per_day.index, weekly_avg_per_day.values, marker='o', color='blue', linewidth=2)

for i, value in enumerate(weekly_avg_per_day.values):
    plt.text(weekly_avg_per_day.index[i], value + 0.5, f"{value:.1f}", ha='center', va='bottom', fontsize=8)

plt.xlabel("Settimana")
plt.ylabel("Media giornaliera quantità ordinata")
plt.title("Media giornaliera della domanda per settimana")
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


# 5. Studio numero di ordini

In [ ]:
# Raggruppa i dati per giorno della settimana e conta il numero di ordini
# Reindex serve per ordinare i giorni da Lun a Ven
weekday_count = delivery_data.groupby('weekday_name')['quantity'].count().reindex(['Lun','Mar','Mer','Gio','Ven'])

# Imposta la dimensione del grafico
plt.figure(figsize=(10,5))

# Crea il grafico a barre con colore salmon e bordo nero
bars = plt.bar(weekday_count.index, weekday_count.values, color='salmon', edgecolor='black')

# Aggiunge il valore numerico sopra ogni barra
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, str(int(bar.get_height())), 
             ha='center', va='bottom', fontsize=10)

# Etichetta asse X
plt.xlabel("Giorno della settimana")

# Etichetta asse Y
plt.ylabel("Numero di ordini")

# Titolo del grafico
plt.title("Numero di ordini per giorno della settimana")

# Mostra il grafico
plt.show()



In [ ]:
# --- Crea colonna settimana ISO (1-52/53) ---
delivery_data['week'] = delivery_data['delivery_date'].dt.isocalendar().week

# --- Numero totale di ordini per settimana ---
# Qui contiamo semplicemente il numero di ordini in ogni settimana
weekly_orders_total = delivery_data.resample('W', on='delivery_date')['quantity'].count()

# --- Conta i giorni effettivi con ordini in ogni settimana ---
weekly_days_with_orders = delivery_data.resample('W', on='delivery_date')['delivery_date'].nunique()

# --- Calcola la media giornaliera degli ordini ---
weekly_orders_avg = weekly_orders_total / weekly_days_with_orders

# ------------------- GRAFICO 1: Numero totale di ordini settimanali -------------------
plt.figure(figsize=(12,5))
plt.plot(weekly_orders_total.index, weekly_orders_total.values, marker='o', color='green', linewidth=2)

# Aggiungi etichette sopra i punti
for i, value in enumerate(weekly_orders_total.values):
    plt.text(weekly_orders_total.index[i], value + 0.5, str(int(value)),
             ha='center', va='bottom', fontsize=8)

plt.xlabel("Settimana")
plt.ylabel("Numero totale di ordini")
plt.title("Numero totale di ordini per settimana")
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

# ------------------- GRAFICO 2: Numero medio di ordini giornalieri per settimana -------------------
plt.figure(figsize=(12,5))
plt.plot(weekly_orders_avg.index, weekly_orders_avg.values, marker='o', color='blue', linewidth=2)

# Aggiungi etichette sopra i punti
for i, value in enumerate(weekly_orders_avg.values):
    plt.text(weekly_orders_avg.index[i], value + 0.5, f"{value:.1f}",
             ha='center', va='bottom', fontsize=8)

plt.xlabel("Settimana")
plt.ylabel("Numero medio di ordini giornalieri")
plt.title("Numero medio di ordini giornalieri per settimana")
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 1. Filtra ordini validi
valid_data = delivery_data[delivery_data['is_event'] == True].copy()

# 2. Crea colonna anno-mese (tipo Period)
valid_data['year_month'] = valid_data['delivery_date'].dt.to_period('M')

# 3. Calcola numero ordini totali per giorno
daily_orders = valid_data.groupby('delivery_day').size().reset_index(name='num_orders')
daily_orders['year_month'] = pd.to_datetime(daily_orders['delivery_day']).dt.to_period('M')

# 4. Calcola somma quantità totale per giorno
daily_quantity = valid_data.groupby('delivery_day')['quantity'].sum().reset_index(name='total_quantity')
daily_quantity['year_month'] = pd.to_datetime(daily_quantity['delivery_day']).dt.to_period('M')

# 5. Calcola giorni attivi per mese (giorni con almeno un ordine)
active_days_per_month = daily_orders.groupby('year_month')['delivery_day'].count()

# 6. Calcola ordini totali per mese
total_orders_per_month = daily_orders.groupby('year_month')['num_orders'].sum()

# 7. Calcola quantità totale per mese
total_quantity_per_month = daily_quantity.groupby('year_month')['total_quantity'].sum()

# 8. Calcola media ordini per giorno attivo
avg_orders_per_day = total_orders_per_month / active_days_per_month

# 9. Calcola media quantità per giorno attivo
avg_quantity_per_day = total_quantity_per_month / active_days_per_month

# 10. Calcola deviazione standard, varianza, CV dei numeri di ordini giornalieri per mese
std_orders = daily_orders.groupby('year_month')['num_orders'].std()
var_orders = std_orders ** 2
# Usa media ordini per calcolare CV (deviazione standard diviso media)
cv_orders = std_orders / avg_orders_per_day

# 11. Prepara DataFrame con statistiche
stats_df = pd.DataFrame({
    'avg_orders': avg_orders_per_day,
    'avg_quantity': avg_quantity_per_day,
    'std': std_orders,
    'var': var_orders,
    'cv': cv_orders
}).reset_index()

# 12. Ordina per anno-mese in modo lineare
stats_df = stats_df.sort_values('year_month')
months_str = stats_df['year_month'].astype(str)

# 13. Grafico
fig, ax1 = plt.subplots(figsize=(14, 7))

# Barre media ordini
bars = ax1.bar(months_str, stats_df['avg_orders'], color='skyblue', edgecolor='black')
ax1.set_xlabel('Mese')
ax1.set_ylabel('Ordini medi per giorno', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_title('Media ordini e quantità colli per giorno per mese')

# Valori sopra le barre
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 15, f"{height:.1f}", ha='center', va='bottom', fontsize=10)
    
    # Statistiche al centro della barra
    text = (f"CV: {stats_df.iloc[i]['cv']:.2f}\n"
            f"Var: {stats_df.iloc[i]['var']:.1f}\n"
            f"Std: {stats_df.iloc[i]['std']:.1f}")
    ax1.text(bar.get_x() + bar.get_width()/2, height/2, text, ha='center', va='center', fontsize=8, color='black')

# Asse y destro per media quantità colli con linea e punti grandi
ax2 = ax1.twinx()

# Imposta limite asse y destro da 0 a massimo valore più un po' di margine
ax2.set_ylim(bottom=0, top=avg_quantity_per_day.max()*1.1)

ax2.plot(months_str, stats_df['avg_quantity'], marker='o', color='orange', markersize=12, linewidth=2, label='Quantità media colli per giorno')
ax2.set_ylabel('Quantità media colli per giorno', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')
ax2.legend(loc='lower right')


plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches

# Stessa parte di calcolo dati come nel tuo codice (1-13) ...

# Poi per il plot:

fig, ax1 = plt.subplots(figsize=(16, 9))

# Barre media ordini (colore azzurro tenue)
bars = ax1.bar(months_str, stats_df['avg_orders'], color='#89CFF0', edgecolor='black')
ax1.set_xlabel('Mese')
ax1.set_ylabel('Ordini medi per giorno', color='#89CFF0')
ax1.tick_params(axis='y', labelcolor='#89CFF0')
ax1.set_title('Media ordini e quantità colli per giorno per mese')

for i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 0.5, f"{height:.1f}",
             ha='center', va='bottom', fontsize=10, color='#89CFF0')
    
    # Media colli per ordine sotto asse x, scritta verticale
    ax1.text(bar.get_x() + bar.get_width()/2, -30, f"Colli/Ord: {stats_df.iloc[i]['avg_packages_per_order']:.2f}",
             ha='center', va='top', fontsize=9, color='navy', rotation=90)

    # Statistiche ordini (nero) - CV e Var più vicini su file separate
    ax1.text(bar.get_x() + bar.get_width()/2, height/2 + 15, f"CV: {stats_df.iloc[i]['cv_orders']:.2f}", 
             ha='center', va='center', fontsize=8, color='black')
    ax1.text(bar.get_x() + bar.get_width()/2, height/2 + 5, f"Var: {stats_df.iloc[i]['var_orders']:.1f}", 
             ha='center', va='center', fontsize=8, color='black')
    ax1.text(bar.get_x() + bar.get_width()/2, height/2 - 5, f"Std: {stats_df.iloc[i]['std_orders']:.1f}", 
             ha='center', va='center', fontsize=8, color='black')

    # Statistiche quantità (rosso chiaro) più vicine su file separate, sotto centro barra
    ax1.text(bar.get_x() + bar.get_width()/2, height/6 + 10, f"CV: {stats_df.iloc[i]['cv_quantity']:.2f}", 
             ha='center', va='center', fontsize=8, color='#FF6F61')
    ax1.text(bar.get_x() + bar.get_width()/2, height/6, f"Var: {stats_df.iloc[i]['var_quantity']:.1f}", 
             ha='center', va='center', fontsize=8, color='#FF6F61')
    ax1.text(bar.get_x() + bar.get_width()/2, height/6 - 10, f"Std: {stats_df.iloc[i]['std_quantity']:.1f}", 
             ha='center', va='center', fontsize=8, color='#FF6F61')

ax1.set_ylim(bottom=-50)

# Asse y destro per media quantità colli con linea e punti grandi arancio chiaro
ax2 = ax1.twinx()
ax2.plot(months_str, stats_df['avg_quantity'], marker='o', color='#FFA07A', markersize=12, linewidth=2)
ax2.set_ylabel('Quantità media colli per giorno', color='#FFA07A')
ax2.tick_params(axis='y', labelcolor='#FFA07A')
ax2.set_ylim(bottom=0)

# Legenda personalizzata
patch_orders = mpatches.Patch(color='black', label='Statistiche ordini (CV, Var, Std)')
patch_quantity = mpatches.Patch(color='#FF6F61', label='Statistiche quantità (CV, Var, Std)')
ax2.legend(handles=[patch_orders, patch_quantity], loc='lower right')

plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches

# Stessa parte di calcolo dati come nel tuo codice (1-13) ...

# Poi per il plot:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches

# Parte calcolo dati (come nel tuo codice) ...

# Inizio plot
fig, ax1 = plt.subplots(figsize=(16, 9))

bars = ax1.bar(months_str, stats_df['avg_orders'], color='#89CFF0', edgecolor='black')
ax1.set_xlabel('Mese')
ax1.set_ylabel('Ordini medi per giorno', color='#89CFF0')
ax1.tick_params(axis='y', labelcolor='#89CFF0')
ax1.set_title('Media ordini e quantità colli per giorno per mese')

for i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 15, f"{height:.1f}",
             ha='center', va='bottom', fontsize=10, color="#1E0BF3")
    
    # Media colli per ordine in orizzontale, colore rosso
    ax1.text(bar.get_x() + bar.get_width()/2, -30,
             f"Colli/Ord: {stats_df.iloc[i]['avg_packages_per_order']:.2f}",
             ha='center', va='top', fontsize=9, color='red', rotation=0)

    # Statistiche ordini (nero)
    ax1.text(bar.get_x() + bar.get_width()/2, height/2 + 15, f"CV: {stats_df.iloc[i]['cv_orders']:.2f}",
             ha='center', va='center', fontsize=8, color='black')
    ax1.text(bar.get_x() + bar.get_width()/2, height/2 + 5, f"Var: {stats_df.iloc[i]['var_orders']:.1f}",
             ha='center', va='center', fontsize=8, color='black')
    ax1.text(bar.get_x() + bar.get_width()/2, height/2 - 5, f"Std: {stats_df.iloc[i]['std_orders']:.1f}",
             ha='center', va='center', fontsize=8, color='black')

    # Statistiche quantità (arancio)
    ax1.text(bar.get_x() + bar.get_width()/2, height/6 + 10, f"CV: {stats_df.iloc[i]['cv_quantity']:.2f}",
             ha='center', va='center', fontsize=8, color="#480600")
    ax1.text(bar.get_x() + bar.get_width()/2, height/6, f"Var: {stats_df.iloc[i]['var_quantity']:.1f}",
             ha='center', va='center', fontsize=8, color='#480600')
    ax1.text(bar.get_x() + bar.get_width()/2, height/6 - 10, f"Std: {stats_df.iloc[i]['std_quantity']:.1f}",
             ha='center', va='center', fontsize=8, color='#480600')

ax1.set_ylim(bottom=0)

ax2 = ax1.twinx()
ax2.plot(months_str, stats_df['avg_quantity'], marker='o', color='#480600', markersize=12, linewidth=2)
ax2.set_ylabel('Quantità media colli per giorno', color='#480600')
ax2.tick_params(axis='y', labelcolor='#480600')
ax2.set_ylim(bottom=0)

# Legenda personalizzata con spiegazione per colli per ordine
patch_orders = mpatches.Patch(color='black', label='Statistiche ordini (CV, Var, Std)')
patch_quantity = mpatches.Patch(color='#480600', label='Statistiche quantità (CV, Var, Std)')
patch_colli_per_ordine = mpatches.Patch(color='red', label='Colli medi per ordine')
ax2.legend(handles=[patch_orders, patch_quantity, patch_colli_per_ordine], loc='lower right', fontsize='small')

plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# --- 1. Assicuriamoci che 'delivery_date' sia in formato datetime ---
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# --- 2. Creiamo una colonna per Anno-Mese (YYYY-MM) ---
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# --- 3. Raggruppamento per mese ---
monthly_stats = delivery_data.groupby('year_month').agg(
    total_packs=('quantity', 'sum'),   # Totale pacchi nel mese
    total_orders=('quantity', 'count') # Numero di ordini nel mese
).reset_index()

# --- 4. Calcolo della quantità media per ordine ---
monthly_stats['avg_packs_per_order'] = (
    monthly_stats['total_packs'] / monthly_stats['total_orders']
).round(2)

print("\nQuantità media per ordine per mese:")
print(monthly_stats)

# --- 5. Grafico dell'andamento mensile ---
plt.figure(figsize=(14,6))
plt.plot(monthly_stats['year_month'].astype(str), monthly_stats['avg_packs_per_order'],
         marker='o', color='orange', linewidth=2, label='Media pacchi per ordine')

# Aggiungi i valori sopra i punti
for i, val in enumerate(monthly_stats['avg_packs_per_order']):
    plt.text(i, val + 0.1, str(val), ha='center', va='bottom', fontsize=9)

plt.xlabel("Mese")
plt.ylabel("Media pacchi per ordine")
plt.title("Andamento mensile della quantità media per ordine")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.show()

# --- 6. Grafico comparativo: totale pacchi vs totale ordini ---
fig, ax1 = plt.subplots(figsize=(14,6))

ax1.set_xlabel("Mese")
ax1.set_ylabel("Totale pacchi", color='blue')
ax1.bar(monthly_stats['year_month'].astype(str), monthly_stats['total_packs'], 
        color='blue', alpha=0.5, label='Totale pacchi')

ax2 = ax1.twinx()
ax2.set_ylabel("Totale ordini", color='green')
ax2.plot(monthly_stats['year_month'].astype(str), monthly_stats['total_orders'], 
         color='green', marker='o', linewidth=2, label='Totale ordini')

plt.title("Totale pacchi e ordini per mese")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assicuriamoci che 'year_month' sia in formato stringa
monthly_stats['year_month_str'] = monthly_stats['year_month'].astype(str)

# --- Grafico a barre della media pacchi per ordine ---
plt.figure(figsize=(14,6))
bars = plt.bar(
    monthly_stats['year_month_str'], 
    monthly_stats['avg_packs_per_order'], 
    color='skyblue', edgecolor='black'
)

# Aggiungi i valori sopra ogni barra
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2, 
        height + 0.3,  # piccolo offset sopra la barra
        f"{height:.2f}", 
        ha='center', va='bottom', fontsize=10
    )

plt.xlabel("Mese")
plt.ylabel("Media pacchi per ordine")
plt.title("Media pacchi per ordine per mese")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assicuriamoci che la colonna delivery_date sia in formato datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'])

# Estrai anno-mese per raggruppamento mensile
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# Raggruppa per anno-mese e giorno della consegna per contare ordini giornalieri
daily_orders = delivery_data.groupby(['year_month', 'delivery_date']).size().reset_index(name='orders_per_day')

# Calcola la media degli ordini giornalieri per ogni mese    
monthly_avg_orders = daily_orders.groupby('year_month')['orders_per_day'].mean()

# Calcola la varianza e la deviazione standard giornaliera per ogni mese
monthly_var_orders = daily_orders.groupby('year_month')['orders_per_day'].var()
monthly_std_orders = daily_orders.groupby('year_month')['orders_per_day'].std()

# Calcola il coefficiente di variazione
monthly_cv_orders = monthly_std_orders / monthly_avg_orders

# Combina tutto in un dataframe
monthly_stats = pd.DataFrame({
    'year_month': monthly_avg_orders.index.astype(str),
    'avg_orders_per_day': monthly_avg_orders.values,
    'var': monthly_var_orders.values,
    'std': monthly_std_orders.values,
    'cv': monthly_cv_orders.values
})

# Plot con annotazioni
plt.figure(figsize=(14,6))
bars = plt.bar(monthly_stats['year_month'], monthly_stats['avg_orders_per_day'], color='skyblue', edgecolor='black')

for bar, var, cv in zip(bars, monthly_stats['var'], monthly_stats['cv']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, f"{height:.2f}", ha='center', va='bottom', fontsize=14)
    plt.text(bar.get_x() + bar.get_width()/2, height - 15, f"Var: {var:.2f}", ha='center', va='top', fontsize=11)
    plt.text(bar.get_x() + bar.get_width()/2, height - 33, f"CV: {cv:.2f}", ha='center', va='top', fontsize=11)

plt.xlabel("Mese")
plt.ylabel("Numero medio giornaliero di ordini")
plt.title("Media giornaliera di ordini per mese con varianza e coefficiente di variazione")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assicuriamoci che 'delivery_date' sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'])

# Estrai anno-mese per raggruppamento mensile
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# Calcola per ogni mese la media della quantità per ordine
monthly_avg_packs_per_order = delivery_data.groupby('year_month')['quantity'].mean()

# Calcola varianza e deviazione standard della quantità per ordine per mese
monthly_var_packs_per_order = delivery_data.groupby('year_month')['quantity'].var()
monthly_std_packs_per_order = delivery_data.groupby('year_month')['quantity'].std()

# Calcola coefficiente di variazione (cv)
monthly_cv_packs_per_order = monthly_std_packs_per_order / monthly_avg_packs_per_order

# Dataframe con le statistiche
monthly_stats = pd.DataFrame({
    'year_month': monthly_avg_packs_per_order.index.astype(str),
    'avg_packs_per_order': monthly_avg_packs_per_order.values,
    'variance': monthly_var_packs_per_order.values,
    'std_dev': monthly_std_packs_per_order.values,
    'cv': monthly_cv_packs_per_order.values
})

# Plot con annotazioni
plt.figure(figsize=(14,6))
bars = plt.bar(monthly_stats['year_month'], monthly_stats['avg_packs_per_order'], color='lightblue', edgecolor='black')

for bar, var, cv in zip(bars, monthly_stats['variance'], monthly_stats['cv']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.05, f"{height:.2f}", ha='center', va='bottom', fontsize=14)
    plt.text(bar.get_x() + bar.get_width()/2, height - 2, f"Var: {var:.2f}", ha='center', va='top', fontsize=11)
    plt.text(bar.get_x() + bar.get_width()/2, height - 3.8, f"CV: {cv:.2f}", ha='center', va='top', fontsize=11)

plt.xlabel("Mese")
plt.ylabel("Media colli per ordine")
plt.title("Media colli per ordine per mese con varianza e coefficiente di variazione")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assicuriamoci che 'delivery_date' sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'])

# Estrai anno-mese in formato stringa per comodo raggruppamento
delivery_data['year_month_str'] = delivery_data['delivery_date'].dt.to_period('M').astype(str)

plt.figure(figsize=(14,6))
sns.boxplot(x='year_month_str', y='quantity', data=delivery_data, color='lightblue')

plt.xlabel("Mese")
plt.ylabel("Quantità colli per ordine")
plt.title("Distribuzione quantità colli per ordine per mese")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Assicurati che delivery_date sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Trova data minima e massima
min_date = delivery_data['delivery_date'].min()
max_date = delivery_data['delivery_date'].max()

# Calcola numero totale di giorni
total_days = (max_date - min_date).days + 1

# Numero di settimane complete
complete_weeks = total_days // 7

# Giorni extra (parziali)
extra_days = total_days % 7

print(f"Totale giorni nel dataset: {total_days}")
print(f"Settimane complete: {complete_weeks}")
print(f"Giorni extra (parziali): {extra_days}")

# --- Conta quante volte compare ogni giorno della settimana ---
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek

# Conta i giorni distinti per weekday
days_per_weekday = delivery_data.drop_duplicates(subset=['delivery_day', 'weekday_num'])
days_count = days_per_weekday['weekday_num'].value_counts().sort_index()

# Mappa numeri ai nomi dei giorni
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven', 5:'Sab', 6:'Dom'}
days_count.index = days_count.index.map(weekday_map)

print("\nNumero di giorni per weekday:")
print(days_count)

# --- Verifica: totale giorni calcolati dai weekday ---
total_weekdays_count = days_count.sum()
print(f"\nSomma giorni per weekday: {total_weekdays_count} (dovrebbe essere uguale a totale_days)")

import pandas as pd

# Assicuriamoci che la colonna sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Trova la data minima e massima
start_date = delivery_data['delivery_date'].min()
end_date = delivery_data['delivery_date'].max()

print(f"Data di inizio del dataset: {start_date.strftime('%Y-%m-%d')}")
print(f"Data di fine del dataset: {end_date.strftime('%Y-%m-%d')}")


In [ ]:
# --- Creiamo una colonna per il periodo mese-anno ---
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# --- Raggruppa per periodo (mese-anno) e conta numero di ordini totali ---
monthly_orders = delivery_data.groupby('year_month')['quantity'].count()

# --- Conta i giorni effettivi presenti in ciascun mese ---
monthly_days = delivery_data.groupby('year_month')['delivery_day'].nunique()

# --- Calcolo media giornaliera di ordini ---
monthly_avg_orders = monthly_orders / monthly_days

# --- Grafico 1: Numero totale di ordini ---
plt.figure(figsize=(12,5))
plt.plot(monthly_orders.index.astype(str), monthly_orders.values, marker='o', color='skyblue', linewidth=2, label="Totale ordini")

# Aggiungi i valori sopra i punti
for i, val in enumerate(monthly_orders.values):
    plt.text(i, val + 0.5, str(int(val)), ha='center', va='bottom', fontsize=9)

plt.xlabel("Periodo (Anno-Mese)")
plt.ylabel("Numero di ordini")
plt.title("Andamento del numero di ordini nel tempo")
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.show()

# --- Grafico 2: Media giornaliera di ordini ---
plt.figure(figsize=(12,5))
plt.plot(monthly_avg_orders.index.astype(str), monthly_avg_orders.values, marker='o', color='orange', linewidth=2, label="Media giornaliera")

# Aggiungi i valori sopra i punti
for i, val in enumerate(monthly_avg_orders.values):
    plt.text(i, val + 0.5, f"{val:.1f}", ha='center', va='bottom', fontsize=9)

plt.xlabel("Periodo (Anno-Mese)")
plt.ylabel("Numero medio giornaliero di ordini")
plt.title("Andamento della media giornaliera di ordini (normalizzata per giorni effettivi)")
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.show()


# 6. MEDIA della domanda totale

In [ ]:
# Crea un grafico a barre con i valori della media della quantità per ogni giorno
# Crea una nuova colonna 'day_of_week' con i nomi in italiano ordinati, partendo da weekday_num
weekday_map_ita = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven'}
delivery_data['day_of_week'] = delivery_data['weekday_num'].map(weekday_map_ita)

# Ora puoi fare il raggruppamento usando questa nuova colonna
weekday_mean_total = delivery_data.groupby('day_of_week')['quantity'].mean().reindex(
    ['Lun', 'Mar', 'Mer', 'Gio', 'Ven']
)


# Grafico a barre
plt.figure(figsize=(10,5))
bars = plt.bar(weekday_mean_total.index, weekday_mean_total.values, color='lightgreen', edgecolor='black')

# Aggiungi valori sopra ogni barra
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.2, f"{height:.1f}", ha='center', va='bottom', fontsize=10)

plt.xlabel("Giorno della settimana")
plt.ylabel("Media quantità ordinata")
plt.title("Media della domanda per giorno della settimana")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calcola media, varianza e deviazione standard per giorno della settimana
# Usa i nomi in italiano per riordinare
weekday_stats = delivery_data.groupby('day_of_week')['quantity'].agg(['mean', 'var', 'std']).reindex(
    ['Lun', 'Mar', 'Mer', 'Gio', 'Ven']
)

# Calcola coefficiente di variazione (cv = std / mean)
weekday_stats['cv'] = weekday_stats['std'] / weekday_stats['mean']

# Grafico a barre della media
plt.figure(figsize=(10, 5))
bars = plt.bar(weekday_stats.index, weekday_stats['mean'], color='lightgreen', edgecolor='black')

# Aggiungi annotazioni sopra le barre per media, varianza e cv
for bar, var, cv in zip(bars, weekday_stats['var'], weekday_stats['cv']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.2, f"{height:.1f}", ha='center', va='bottom', fontsize=14)
    plt.text(bar.get_x() + bar.get_width() / 2, height - 1.5, f"Var: {var:.2f}", ha='center', va='top', fontsize=11)
    plt.text(bar.get_x() + bar.get_width() / 2, height - 3.5, f"CV: {cv:.2f}", ha='center', va='top', fontsize=11)

plt.xlabel("Giorno della settimana")
plt.ylabel("Media quantità ordinata")
plt.title("Media, varianza e coefficiente di variazione per giorno della settimana")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

order = ['Lun', 'Mar', 'Mer', 'Gio', 'Ven']

plt.figure(figsize=(10, 6))
sns.boxplot(x='day_of_week', y='quantity', data=delivery_data, order=order, color='lightgreen')

plt.xlabel("Giorno della settimana")
plt.ylabel("Quantità ordinata")
plt.title("Distribuzione quantità ordinata per giorno della settimana")
plt.show()


In [ ]:
#media della domanda totale per ogni cliente per ogni giorno della settimana per mese
delivery_data['day_of_week'] = delivery_data['delivery_date'].dt.day_name()
# Mese come numero o nome
delivery_data['month'] = delivery_data['delivery_date'].dt.month_name()  # es. "September"
monthly_weekday_avg = delivery_data.groupby(
    ['location_id', 'month', 'day_of_week']
)['quantity'].mean().reset_index()
monthly_weekday_pivot = monthly_weekday_avg.pivot_table(
    index=['location_id', 'month'],
    columns='day_of_week',
    values='quantity'
)
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
monthly_weekday_pivot = monthly_weekday_pivot[days_order]
print(monthly_weekday_pivot.head(10))  # prime 10 location_id e mesi


In [ ]:
# Crea colonna giorno della settimana
delivery_data['day_of_week'] = delivery_data['delivery_date'].dt.day_name()

# Crea colonna mese (nome del mese)
delivery_data['month'] = delivery_data['delivery_date'].dt.month_name()

# Raggruppa per mese e giorno della settimana e calcola la media aggregata su tutti i clienti
monthly_weekday_total_avg = delivery_data.groupby(
    ['month', 'day_of_week']
)['quantity'].mean().reset_index()

# Pivot per avere i giorni della settimana come colonne
monthly_weekday_total_pivot = monthly_weekday_total_avg.pivot(
    index='month', 
    columns='day_of_week', 
    values='quantity'
)

# Ordina le colonne dei giorni della settimana
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
monthly_weekday_total_pivot = monthly_weekday_total_pivot[days_order]

# Ordina i mesi in ordine naturale (opzionale)
months_order = [
    'June',
    'July','August','September','October','November','December', 'January']
monthly_weekday_total_pivot = monthly_weekday_total_pivot.reindex(months_order)

# Mostra il risultato
print(monthly_weekday_total_pivot.head(10))  # prime 10 mesi (se presenti)


# --- Creazione del grafico a barre raggruppate ---
plt.figure(figsize=(14, 6))

# Ottieni i mesi e i giorni ordinati
mesi = monthly_weekday_total_pivot.index
giorni = monthly_weekday_total_pivot.columns

# Larghezza di ogni barra
bar_width = 0.15

# Posizioni di base sull'asse X
x = np.arange(len(mesi))

# Disegna una barra per ogni giorno della settimana
for i, giorno in enumerate(giorni):
    plt.bar(x + i * bar_width, 
            monthly_weekday_total_pivot[giorno], 
            width=bar_width, 
            label=giorno)

# Etichette e titolo
plt.xlabel("Mese")
plt.ylabel("Domanda media (quantity)")
plt.title("Domanda media per giorno della settimana e mese (tutti i clienti)")
plt.xticks(x + bar_width * 2, mesi, rotation=45)  # Centra le etichette dei mesi
plt.legend(title="Giorno della settimana")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostra il grafico
plt.tight_layout()
plt.show()


In [ ]:
# --- Grafico per esempio: media quantità per giorni della settimana per un mese ---
# Seleziona un mese, ad esempio "September"
month_to_plot = 'September'
plt.figure(figsize=(10,5))
bars = plt.bar(
    monthly_weekday_total_pivot.loc[month_to_plot].index, 
    monthly_weekday_total_pivot.loc[month_to_plot].values, 
    color='lightblue', edgecolor='black'
)

# Aggiungi valori sopra ogni barra
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.2, f"{height:.1f}", 
             ha='center', va='bottom', fontsize=10)

plt.xlabel("Giorno della settimana")
plt.ylabel("Media quantità ordinata")
plt.title(f"Media domanda per giorno della settimana - {month_to_plot}")
plt.show()

# COUNT delle consegne per singolo cliente che cadono in un giorno della settimana

In [ ]:
# Conta ordini per cliente e giorno della settimana
weekday_counts = delivery_data.groupby(['location_id', 'day_of_week']).size().reset_index(name='count')
# Pivot per avere giorni come colonne
weekday_counts_pivot = weekday_counts.pivot(index='location_id', columns='day_of_week', values='count')
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekday_counts_pivot = weekday_counts_pivot[days_order]
weekday_counts_pivot = weekday_counts_pivot.fillna(0).astype(int)
print(weekday_counts_pivot.head(10))  # prime 10 location_id

# 7. ANALISI DEI CLIENTI CHE ORDINANO IL SABATO

In [ ]:
import pandas as pd

# --- 1. Carica i dati ---
delivery_data = pd.read_csv('delivery_history.csv').copy()

# --- 2. Conversione variabile delivery_date in datetime ---
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# --- 3. Creazione di colonne derivate dalla data ---
# Giorno della settimana: 0=Monday, 6=Sunday
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek

# Mappa i numeri ai nomi (Lun=0, ..., Dom=6)
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven', 5:'Sab', 6:'Dom'}
delivery_data['weekday_name'] = delivery_data['weekday_num'].map(weekday_map)

# --- 4. Conversione is_event in booleano e filtro solo eventi validi ---
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

# --- 5. Filtra solo clienti che hanno almeno una consegna di sabato ---
saturday_customers = delivery_data[delivery_data['weekday_name'] == 'Sab']['location_id'].unique()

# Filtra il dataset solo per questi clienti
sat_customers_data = delivery_data[delivery_data['location_id'].isin(saturday_customers)].copy()

# --- 6. Conta le consegne per cliente e per giorno della settimana ---
# Pivot table: righe = cliente, colonne = giorni della settimana
deliveries_per_day = sat_customers_data.pivot_table(
    index='location_id',
    columns='weekday_name',
    values='delivery_date',
    aggfunc='count',
    fill_value=0
)

# Aggiungi colonna numero totale di consegne
deliveries_per_day['totale_consegne'] = deliveries_per_day.sum(axis=1)

# Calcola percentuale consegne di sabato
deliveries_per_day['percentuale_sabato'] = deliveries_per_day['Sab'] / deliveries_per_day['totale_consegne'] * 100

# Riordina le colonne in ordine: Lun, Mar, Mer, Gio, Ven, Sab, Dom, Totale, Percentuale
cols_order = ['Lun','Mar','Mer','Gio','Ven','Sab','Dom','totale_consegne','percentuale_sabato']
deliveries_per_day = deliveries_per_day.reindex(columns=cols_order)

# --- 7. Mostra il risultato ---
print(deliveries_per_day)


In [ ]:
delivery_data = pd.read_csv('delivery_history.csv').copy()
#2. conversione variabile delivery_date in datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')
# Crea una seconda colonna solo con la data, tipo object (utile solo per visualizzazioni)
delivery_data['delivery_day'] = delivery_data['delivery_date'].dt.date

#3. Creazione di colonne derivate dalla data
#Giorno della settimana: 0=Monday, 6=Sunday
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek
#Escludi sabati e domeniche
# weekday_num: 0=Lun, 1=Mar, ..., 5=Sab, 6=Dom
delivery_data = delivery_data[delivery_data['weekday_num'] < 5]
delivery_data = delivery_data.dropna(subset=['delivery_day'])

#4. Mappa i numeri ai nomi (utile per grafici)
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven'}
delivery_data['weekday_name'] = delivery_data['weekday_num'].map(weekday_map)

#5. Divisione delle singole variabili temporali
# Mese (1-12)
delivery_data['month'] = delivery_data['delivery_date'].dt.month
# Anno
delivery_data['year'] = delivery_data['delivery_date'].dt.year
# Periodo (Anno-Mese) utile per analisi mensili
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

#6. Conversione is_event in booleano
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
# Applica conversione solo ai validi (opzionale: puoi forzare tutto a bool, ma questo è più sicuro)
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

# --- Check finale ---
print(delivery_data[['delivery_date', 'weekday_name', 'month', 'year', 'year_month', 'is_event']].head())


In [ ]:
import pandas as pd

# Assumiamo che delivery_data sia già filtrato e contenga solo is_event=True

# Crea colonna giorno della settimana
delivery_data['weekday_name'] = delivery_data['delivery_date'].dt.day_name()

# Filtra solo sabato
saturday = 'Saturday'

# Raggruppa per cliente e giorno della settimana e conta le consegne
weekly_counts = delivery_data.groupby(['location_id', 'weekday_name']).size().unstack(fill_value=0)

# Calcola il totale delle consegne per ogni cliente
weekly_counts['total_deliveries'] = weekly_counts.sum(axis=1)

# Calcola la percentuale di consegne di sabato
if saturday in weekly_counts.columns:
    weekly_counts['perc_saturday'] = weekly_counts[saturday] / weekly_counts['total_deliveries'] * 100
else:
    weekly_counts[saturday] = 0
    weekly_counts['perc_saturday'] = 0

# --- Clienti con almeno 2 consegne di sabato ---
clients_saturday_2plus = weekly_counts[weekly_counts[saturday] >= 2]
print("Clienti con almeno 2 consegne di sabato:")
print(clients_saturday_2plus)

# --- Clienti che hanno consegne solo di sabato ---
other_days = [d for d in weekly_counts.columns if d not in ['total_deliveries', 'perc_saturday', saturday]]
clients_only_saturday = weekly_counts[(weekly_counts[saturday] > 0) & (weekly_counts[other_days].sum(axis=1) == 0)]
print("\nClienti con consegne solo di sabato:")
print(clients_only_saturday)


In [ ]:
import pandas as pd

# Assicuriamoci che la colonna delivery_date sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Creiamo una colonna con il giorno della settimana (0=Lun, 6=Dom)
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek

# Raggruppiamo per giorno della settimana e contiamo il numero totale di ordini
orders_per_weekday = delivery_data.groupby('weekday_num')['quantity'].count()

# Mappa i numeri ai nomi dei giorni
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven', 5:'Sab', 6:'Dom'}
orders_per_weekday.index = orders_per_weekday.index.map(weekday_map)

# Ordini totali per giorno della settimana
print("\nNumero totale di ordini per giorno della settimana:")
print(orders_per_weekday)

# --- Opzionale: grafico a barre ---
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
bars = plt.bar(orders_per_weekday.index, orders_per_weekday.values, color='skyblue', edgecolor='black')

# Aggiungi i valori sopra le barre
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 2, str(int(height)), ha='center', va='bottom')

plt.xlabel("Giorno della settimana")
plt.ylabel("Numero totale di ordini")
plt.title("Numero totale di ordini per giorno della settimana")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assicuriamoci che delivery_date sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Giorno della settimana: 0=Lun, 6=Dom
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek

# Raggruppiamo per giorno della settimana
orders_per_weekday = delivery_data.groupby('weekday_num')['quantity'].count()
total_packs_per_weekday = delivery_data.groupby('weekday_num')['quantity'].sum()

# Mappa i numeri ai nomi dei giorni
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven', 5:'Sab', 6:'Dom'}
orders_per_weekday.index = orders_per_weekday.index.map(weekday_map)
total_packs_per_weekday.index = total_packs_per_weekday.index.map(weekday_map)

# --- Grafico con doppio asse Y ---
fig, ax1 = plt.subplots(figsize=(12,6))

# Barre per numero di ordini (asse Y sinistro)
bars = ax1.bar(orders_per_weekday.index, orders_per_weekday.values, color='skyblue', alpha=0.7, label='Ordini totali')
ax1.set_xlabel("Giorno della settimana")
ax1.set_ylabel("Numero totale di ordini", color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')

# Linea per quantità totale pacchi (asse Y destro)
ax2 = ax1.twinx()
ax2.plot(total_packs_per_weekday.index, total_packs_per_weekday.values, color='orange', marker='o', linewidth=2, label='Quantità totale pacchi')
ax2.set_ylabel("Quantità totale pacchi", color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

# Valori sopra le barre
for i, val in enumerate(orders_per_weekday.values):
    ax1.text(i, val + 5, str(int(val)), ha='center', va='bottom', fontsize=9)

plt.title("Ordini totali e quantità totale di pacchi per giorno della settimana")
fig.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assicuriamoci che delivery_date sia datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Aggiungiamo colonne per anno e mese
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# --- Raggruppiamo per mese ---
monthly_stats = delivery_data.groupby('year_month').agg(
    total_orders=('quantity', 'count'),
    total_packs=('quantity', 'sum'),
    days_in_month=('delivery_day', 'nunique')  # conta i giorni distinti con dati
).reset_index()

# Calcola media per giorno del mese
monthly_stats['avg_orders_per_day'] = monthly_stats['total_orders'] / monthly_stats['days_in_month']
monthly_stats['avg_packs_per_day'] = monthly_stats['total_packs'] / monthly_stats['days_in_month']

# Converti year_month in stringa per il plot
monthly_stats['year_month_str'] = monthly_stats['year_month'].astype(str)

print(monthly_stats[['year_month_str', 'avg_orders_per_day', 'avg_packs_per_day']])

# --- Grafico con doppio asse Y ---
fig, ax1 = plt.subplots(figsize=(14,6))

# Barre per numero medio di ordini per giorno
bars = ax1.bar(monthly_stats['year_month_str'], monthly_stats['avg_orders_per_day'], 
               color='skyblue', alpha=0.7, label='Ordini medi per giorno')
ax1.set_xlabel("Mese")
ax1.set_ylabel("Ordini medi per giorno", color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')

# Linea per quantità media di pacchi per giorno
ax2 = ax1.twinx()
ax2.plot(monthly_stats['year_month_str'], monthly_stats['avg_packs_per_day'], 
         color='orange', marker='o', linewidth=2, label='Quantità media pacchi per giorno')
ax2.set_ylabel("Quantità media pacchi per giorno", color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

# Valori sopra le barre
for i, val in enumerate(monthly_stats['avg_orders_per_day']):
    ax1.text(i, val + 15, f"{val:.1f}", ha='center', va='bottom', fontsize=9)

plt.title("Media ordini e pacchi per giorno per mese")
plt.xticks(rotation=45)
fig.tight_layout()
plt.show()
